# # Network_Threat_Analysis_System
# ## Comprehensive Feature Extraction and Machine Learning Framework
# 
# This notebook integrates three analysis approaches:
# 1. Statistical flow analysis
# 2. Semantic content analysis  
# 3. Visual pattern generation
#
# Output includes both human-readable reports and ML-ready datasets.


# ### Step 1: Environment Setup and Dependency Installation

# ### Step 1:  Setup and Imports


In [1]:
# Cell 1: Setup and Imports - Complete Environment Initialization
"""
PURPOSE: Initialize the complete analysis environment with all required libraries
This cell sets up the entire working environment for network packet analysis.

WHAT THIS CELL DOES:
1. Imports all necessary Python libraries for:
   - Network packet parsing (scapy, pyshark)
   - Data processing (pandas, numpy)
   - Machine learning (sklearn, xgboost, lightgbm)
   - Visualization (plotly, matplotlib, seaborn)
   - System monitoring (psutil for memory tracking)
   - File operations (parquet for efficient storage, no HDF5)
2. Configures display settings for better readability
3. Shows system information (total RAM, available memory)
4. Timestamps when analysis starts

WHY THESE LIBRARIES:
- scapy/pyshark: Parse PCAP files and extract packet information
- pandas/numpy: Handle large datasets efficiently with DataFrames
- sklearn: Provides ML algorithms and feature selection tools
- SGDClassifier: Enables incremental learning for large datasets
- xgboost/lightgbm: High-performance gradient boosting for classification
- plotly: Creates interactive visualizations
- psutil: Monitors memory usage to prevent crashes
- pyarrow: Enables Parquet storage without file locking issues on Windows

IMPORTANT CHANGES:
- Removed h5py/HDF5 (unreliable on Windows)
- Added pyarrow for Parquet support (better compression, no locking)
- Added lightgbm as alternative to xgboost
"""

import os
import gc
import warnings
import json
import pickle
import hashlib
import re
import psutil
import glob
import time
from datetime import datetime
from collections import defaultdict, Counter, deque
from typing import Dict, List, Tuple, Optional, Any, Generator
from dataclasses import dataclass, field

# Data processing libraries
import numpy as np
import pandas as pd
from scipy import stats

# Parquet support (replacing HDF5)
import pyarrow as pa
import pyarrow.parquet as pq

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier  # For incremental learning when data doesn't fit in memory
from sklearn.cluster import MiniBatchKMeans

# Gradient boosting libraries
import xgboost as xgb
import lightgbm as lgb  # Often faster than XGBoost on CPU

# Network analysis libraries
from scapy.all import *  # For packet parsing
import pyshark  # Alternative packet parser with better protocol support

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.subplots import make_subplots

# Jupyter notebook specific
from IPython.display import display, clear_output, HTML
from tqdm.notebook import tqdm  # Progress bars for loops

# Interactive widgets for configuration
import ipywidgets as widgets

# Parallel processing
from joblib import Parallel, delayed
import multiprocessing

# Global configuration
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output
pd.set_option('display.max_columns', 50)  # Show more columns in DataFrame display
pd.set_option('display.max_rows', 100)  # Show more rows in DataFrame display

# Display system information
print("="*70)
print("MEMORY-OPTIMIZED NETWORK ANALYSIS PIPELINE V2.0")
print("="*70)
print(f"System Memory: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"Available Memory: {psutil.virtual_memory().available / (1024**3):.1f} GB")
print(f"CPU Cores: {psutil.cpu_count()} ({multiprocessing.cpu_count()} usable)")
print(f"Storage Backend: Parquet (no file locking issues)")
print(f"Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

MEMORY-OPTIMIZED NETWORK ANALYSIS PIPELINE V2.0
System Memory: 31.4 GB
Available Memory: 14.8 GB
CPU Cores: 12 (12 usable)
Storage Backend: Parquet (no file locking issues)
Analysis started at: 2025-08-30 04:46:19


# ### Step 1.5: Clear Temporary Files Before Analysis 

In [ ]:
# Cell 1.5: Cleanup - Clear Temporary Files Before Analysis
"""
PURPOSE: Clean up temporary files from previous runs
This cell ensures a clean start by removing old temporary files that could
interfere with the current analysis.

WHAT THIS CELL DOES:
1. Checks for existing temporary directories
2. Removes old flow, semantic, combined, and labeled features
3. Clears any cached files from previous runs
4. Recreates clean directory structure
5. Optionally clears output directory

WHY THIS MATTERS:
- Prevents mixing data from different PCAP files
- Avoids file naming conflicts
- Ensures consistent results
- Frees up disk space
- Prevents loading old data accidentally

RUN THIS:
- Before starting a new analysis
- After a failed/interrupted run
- When switching between different PCAP files
"""

import os
import shutil
import psutil

def cleanup_temp_files(clear_output=False):
    """
    Remove temporary files from previous pipeline runs.
    
    Args:
        clear_output: If True, also clears the output directory (default: False)
    """
    print("="*60)
    print("CLEANING UP TEMPORARY FILES")
    print("="*60)
    
    # Check if Config is defined
    try:
        temp_dir = Config.TEMP_DIR
        output_dir = Config.OUTPUT_DIR
    except NameError:
        # Use default values if Config not yet defined
        temp_dir = './temp_processing'
        output_dir = './analysis_output'
        print("⚠ Config not defined, using default directories")
    
    # List of temporary directories to clean
    temp_dirs = [
        os.path.join(temp_dir, 'flow_features'),
        os.path.join(temp_dir, 'semantic_features'),
        os.path.join(temp_dir, 'combined_features'),
        os.path.join(temp_dir, 'labeled_features')
    ]
    
    # Clean each temporary directory
    for dir_path in temp_dirs:
        if os.path.exists(dir_path):
            try:
                # Count files before deletion
                file_count = sum(len(files) for _, _, files in os.walk(dir_path))
                size_mb = sum(os.path.getsize(os.path.join(dirpath, filename))
                             for dirpath, _, filenames in os.walk(dir_path)
                             for filename in filenames) / (1024*1024)
                
                # Remove directory
                shutil.rmtree(dir_path)
                print(f"✓ Removed: {os.path.basename(dir_path)}")
                print(f"  Deleted {file_count} files ({size_mb:.1f} MB)")
            except Exception as e:
                print(f"✗ Failed to remove {dir_path}: {e}")
    
    # Clean root temp directory if empty or recreate it
    if os.path.exists(temp_dir):
        try:
            # Check if temp directory has any other files
            remaining_files = os.listdir(temp_dir)
            if not remaining_files:
                shutil.rmtree(temp_dir)
                print(f"✓ Removed empty temp directory")
        except Exception as e:
            print(f"⚠ Could not clean temp directory: {e}")
    
    # Recreate clean directory structure
    os.makedirs(temp_dir, exist_ok=True)
    for dir_name in ['flow_features', 'semantic_features', 'combined_features', 'labeled_features']:
        os.makedirs(os.path.join(temp_dir, dir_name), exist_ok=True)
    print("✓ Created clean directory structure")
    
    # Optionally clear output directory
    if clear_output and os.path.exists(output_dir):
        try:
            # Ask for confirmation
            response = input(f"\n⚠ Clear output directory '{output_dir}'? (y/n): ")
            if response.lower() == 'y':
                shutil.rmtree(output_dir)
                os.makedirs(output_dir, exist_ok=True)
                print(f"✓ Cleared output directory")
            else:
                print("✓ Output directory preserved")
        except Exception as e:
            print(f"✗ Failed to clear output directory: {e}")
    
    # Report disk space
    disk = psutil.disk_usage('/')
    print(f"\nDisk space available: {disk.free / (1024**3):.1f} GB ({100-disk.percent:.1f}% free)")
    
    print("\n✓ Cleanup complete - ready for new analysis")
    print("="*60)

def check_temp_files():
    """
    Check what temporary files currently exist without deleting them.
    """
    print("="*60)
    print("CHECKING TEMPORARY FILES")
    print("="*60)
    
    # Check if Config is defined
    try:
        temp_dir = Config.TEMP_DIR
    except NameError:
        temp_dir = './temp_processing'
        print("⚠ Config not defined, checking default directory")
    
    if not os.path.exists(temp_dir):
        print("No temporary directory found - clean start")
        return
    
    # Check each subdirectory
    total_files = 0
    total_size = 0
    
    for root, dirs, files in os.walk(temp_dir):
        if files:
            dir_size = sum(os.path.getsize(os.path.join(root, f)) for f in files)
            total_files += len(files)
            total_size += dir_size
            
            rel_path = os.path.relpath(root, temp_dir)
            print(f"\n{rel_path}:")
            print(f"  Files: {len(files)}")
            print(f"  Size: {dir_size / (1024*1024):.1f} MB")
            
            # Show sample files
            sample_files = files[:3]
            for f in sample_files:
                print(f"    - {f}")
            if len(files) > 3:
                print(f"    ... and {len(files)-3} more files")
    
    if total_files > 0:
        print(f"\nTotal: {total_files} files, {total_size / (1024*1024):.1f} MB")
        print("\n⚠ Temporary files exist from previous run")
        print("  Run cleanup_temp_files() to remove them")
    else:
        print("\n✓ No temporary files found - clean start")
    
    print("="*60)

# Create interactive cleanup interface
from IPython.display import display
import ipywidgets as widgets

def create_cleanup_ui():
    """
    Create an interactive cleanup interface with buttons.
    """
    display(HTML("<h3>Cleanup Manager</h3>"))
    
    # Output area for messages
    output = widgets.Output()
    
    # Check button
    check_btn = widgets.Button(
        description='Check Temp Files',
        button_style='info',
        tooltip='Check what temporary files exist',
        icon='search'
    )
    
    # Clean temp button
    clean_temp_btn = widgets.Button(
        description='Clean Temp Files',
        button_style='warning',
        tooltip='Remove temporary processing files',
        icon='trash'
    )
    
    # Clean all button
    clean_all_btn = widgets.Button(
        description='Clean Everything',
        button_style='danger',
        tooltip='Remove all temporary and output files',
        icon='exclamation-triangle'
    )
    
    def on_check(b):
        with output:
            clear_output()
            check_temp_files()
    
    def on_clean_temp(b):
        with output:
            clear_output()
            cleanup_temp_files(clear_output=False)
    
    def on_clean_all(b):
        with output:
            clear_output()
            cleanup_temp_files(clear_output=True)
    
    check_btn.on_click(on_check)
    clean_temp_btn.on_click(on_clean_temp)
    clean_all_btn.on_click(on_clean_all)
    
    # Layout
    button_box = widgets.HBox([check_btn, clean_temp_btn, clean_all_btn])
    
    display(widgets.VBox([
        widgets.HTML("<p>Manage temporary files from previous runs:</p>"),
        button_box,
        output
    ]))

# Display the cleanup UI
create_cleanup_ui()

print("\n" + "="*60)
print("QUICK COMMANDS:")
print("="*60)
print("# Check what files exist:")
print("check_temp_files()")
print("")
print("# Clean only temp files (recommended before each run):")
print("cleanup_temp_files()")
print("")
print("# Clean everything including output:")
print("cleanup_temp_files(clear_output=True)")
print("="*60)


QUICK COMMANDS:
# Check what files exist:
check_temp_files()

# Clean only temp files (recommended before each run):
cleanup_temp_files()

# Clean everything including output:
cleanup_temp_files(clear_output=True)


# ### Step 2: Configuration Interface

In [ ]:
# Cell 2: Configuration Class - Interactive Settings & Memory Management
"""
PURPOSE: Central configuration hub for all analysis parameters
This cell creates an interactive configuration interface using Jupyter widgets.

WHAT THIS CELL DOES:
1. Defines all configurable parameters for the analysis pipeline
2. Creates an interactive UI with sliders, text boxes, and checkboxes
3. Automatically determines optimal processing strategy based on file size
4. Validates user inputs and checks system resources
5. Sets up directory structure for outputs and temporary files

KEY FUNCTIONALITY:
- File Management: Handles PCAP input files and CICIDS label CSV files
- Memory Management: Sets limits on RAM usage and flow storage
- Processing Strategy: Chooses between disk-based or memory-based processing
- ML Configuration: Selects which models to train and sampling strategy
- Feature Settings: Determines number of features and data types to use

CONFIGURATION CATEGORIES:
1. Input/Output: PCAP file, label files, output directory
2. Memory Settings: MAX_MEMORY_GB, MAX_FLOWS_IN_MEMORY, batch sizes
3. Processing: Chunk sizes, flow timeouts, packet limits
4. Analysis Modes: Flow analysis, semantic analysis, NLP depth
5. ML Settings: Model selection, train/test split, sampling
6. Storage: Parquet format usage, compression, temporary file cleanup

IMPORTANT CHANGES:
- Replaced HDF5 with Parquet storage (more reliable on Windows)
- Increased default memory limits for 32GB system
- Added LightGBM to model selection
- Optimized chunk sizes for AMD Ryzen CPU

The class uses class methods (@classmethod) so configuration is global
and accessible throughout the entire pipeline.
"""

class Config:
    # ============= FILE PATHS =============
    PCAP_FILE = ''  # Path to input PCAP file (10+ GB for CICIDS2017)
    LABEL_FILE = ''  # Single label file path (will be set if multiple CSVs combined)
    LABEL_FILES = []  # List of CICIDS2017 CSV label files
    OUTPUT_DIR = './analysis_output'  # Where to save results
    
    # ============= MEMORY OPTIMIZATION (Optimized for 32GB System) =============
    MAX_MEMORY_GB = 8.0  # Maximum RAM to use (25% of 32GB, prevents system freeze)
    MAX_FLOWS_IN_MEMORY = 100000  # Flows kept in RAM before disk flush (increased for 32GB)
    MAX_PACKETS_PER_FLOW = 1000  # Cap packets per flow to prevent memory bloat
    USE_DISK_CACHE = True  # Use Parquet disk storage for large files
    TEMP_DIR = './temp_processing'  # Temporary storage location
    
    # ============= PROCESSING PARAMETERS (Optimized for Ryzen 5) =============
    CHUNK_SIZE = 50000  # Packets processed at once (increased for better CPU utilization)
    BATCH_SIZE = 200000  # Flows per ML training batch (increased for 32GB RAM)
    MAX_PACKETS = 0  # Limit packets to process (0 = unlimited)
    FLOW_TIMEOUT = 120  # Seconds before flow considered complete
    N_JOBS = -1  # Number of parallel jobs (-1 = use all CPU cores)
    
    # ============= SAMPLING STRATEGY =============
    USE_SAMPLING = True  # Sample data if too large for ML
    SAMPLE_SIZE = 1000000  # Maximum flows for ML training (increased for 32GB)
    STRATIFY_SAMPLE = True  # Maintain attack type distribution in sample
    
    # ============= ANALYSIS MODES =============
    ANALYSIS_MODE = 'combined'  # Options: 'flow', 'semantic', 'combined'
    DEEP_INSPECTION = True  # Enable NLP payload analysis (slower but better detection)
    USE_CICIDS_LABELS = False  # Whether to use CICIDS ground truth
    GENERATE_VISUALS = True  # Create visualization charts
    ML_EXPORT = True  # Export ML models and features
    SKIP_ML = False  # Skip ML training for pipeline testing
    
    # ============= FEATURE ENGINEERING =============
    TOP_FEATURES = 50  # Number of best features to select (increased for better accuracy)
    FEATURE_DTYPE = np.float32  # Data type (float32 saves memory vs float64)
    EXTRACT_CICIDS_FEATURES = True  # Extract CICIDS-specific features
    
    # ============= MACHINE LEARNING =============
    TEST_SIZE = 0.2  # Fraction of data for testing
    RANDOM_STATE = 42  # Random seed for reproducibility
    SELECTED_MODELS = ['sgd', 'lightgbm', 'xgboost_incremental']  # Memory-efficient models
    USE_INCREMENTAL_LEARNING = True  # Train in batches for large datasets
    
    # ============= STORAGE SETTINGS (Using Parquet) =============
    STORAGE_FORMAT = 'parquet'  # Storage format (parquet recommended over hdf5)
    USE_PARQUET = True  # Use Parquet format for efficient disk storage
    COMPRESSION = 'snappy'  # Fast compression algorithm
    CLEANUP_TEMP = True  # Delete temporary files after completion
    
    @classmethod
    def check_memory_requirements(cls):
        """
        Analyzes PCAP file size and available RAM to determine processing strategy.
        
        Returns:
            str: 'DISK_BASED' for large files, 'MEMORY_BASED' for small files
        
        This method prevents out-of-memory errors by choosing appropriate
        processing strategy before analysis begins.
        """
        if not cls.PCAP_FILE or not os.path.exists(cls.PCAP_FILE):
            return False
            
        file_size_gb = os.path.getsize(cls.PCAP_FILE) / (1024**3)
        available_gb = psutil.virtual_memory().available / (1024**3)
        total_gb = psutil.virtual_memory().total / (1024**3)
        
        print("\n" + "="*60)
        print("MEMORY ASSESSMENT")
        print("="*60)
        print(f"PCAP Size: {file_size_gb:.2f} GB")
        print(f"Total RAM: {total_gb:.2f} GB")
        print(f"Available RAM: {available_gb:.2f} GB")
        print(f"Max Memory Setting: {cls.MAX_MEMORY_GB:.2f} GB")
        
        # Decision logic for processing strategy
        if file_size_gb > available_gb * 0.3:  # File is >30% of available RAM
            print("\nRECOMMENDATION: Large file detected")
            print("- Using disk-based processing with Parquet")
            print("- Enabling flow timeout mechanism")
            print("- Using incremental learning")
            cls.USE_DISK_CACHE = True
            cls.USE_INCREMENTAL_LEARNING = True
            cls.USE_SAMPLING = True
            processing_strategy = "DISK_BASED"
        else:
            print("\nRECOMMENDATION: File can fit in memory")
            print("- Using hybrid processing")
            processing_strategy = "MEMORY_BASED"
        
        # Optimizations for AMD Ryzen CPU
        cpu_count = psutil.cpu_count()
        print(f"\nCPU Optimization: {cpu_count} threads available")
        cls.N_JOBS = cpu_count - 2  # Leave 2 threads for system
        
        # Time estimation (empirical: 1GB ≈ 2-4 minutes with optimizations)
        estimated_time = file_size_gb * (3 if cls.USE_DISK_CACHE else 2)
        print(f"\nEstimated Processing Time: {estimated_time:.0f} minutes")
        
        return processing_strategy
    
    @classmethod
    def setup_interactive_ui(cls):
        """
        Creates an interactive configuration interface using Jupyter widgets.
        This provides a user-friendly way to set all parameters without
        editing code directly.
        """
        
        display(HTML("<h2>Memory-Optimized Network Analysis Configuration (V2.0)</h2>"))
        
        # ========== MEMORY SETTINGS SECTION ==========
        display(HTML("<h3>Memory Optimization Settings</h3>"))
        
        memory_slider = widgets.FloatSlider(
            value=8.0,
            min=1.0,
            max=psutil.virtual_memory().total / (1024**3),
            step=0.5,
            description='Max RAM (GB):',
            style={'description_width': 'initial'},
            readout_format='.1f'
        )
        
        max_flows = widgets.IntText(
            value=100000,
            description='Max Flows in Memory:',
            tooltip='Flows to keep before flushing to disk',
            style={'description_width': 'initial'}
        )
        
        use_sampling = widgets.Checkbox(
            value=True,
            description='Use Sampling for ML (recommended for large files)',
            indent=False
        )
        
        sample_size = widgets.IntText(
            value=1000000,
            description='Sample Size:',
            disabled=not use_sampling.value,
            style={'description_width': 'initial'}
        )
        
        # Skip ML checkbox for testing
        skip_ml = widgets.Checkbox(
            value=False,
            description='Skip ML Training (for testing pipeline)',
            indent=False,
            style={'description_width': 'initial'}
        )
        
        # Storage format selection
        storage_format = widgets.RadioButtons(
            options=['parquet', 'csv'],
            value='parquet',
            description='Storage Format:',
            style={'description_width': 'initial'}
        )
        
        # Dynamic UI: Enable/disable sample size based on checkbox
        def toggle_sample_size(change):
            sample_size.disabled = not change['new']
        use_sampling.observe(toggle_sample_size, names='value')
        
        # ========== FILE SELECTION SECTION ==========
        pcap_input = widgets.Text(
            placeholder='/path/to/your.pcap',
            description='PCAP File:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='70%')
        )
        
        label_input = widgets.Textarea(
            placeholder='Enter CSV paths (one per line) or leave empty',
            description='Label Files:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='70%', height='80px')
        )
        
        output_dir = widgets.Text(
            value='./analysis_output',
            description='Output Dir:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='70%')
        )
        
        # ========== PROCESSING OPTIONS ==========
        chunk_slider = widgets.IntSlider(
            value=50000,
            min=10000,
            max=100000,
            step=10000,
            description='Chunk Size:',
            continuous_update=False,
            tooltip='Packets processed at once',
            style={'description_width': 'initial'}
        )
        
        batch_slider = widgets.IntSlider(
            value=200000,
            min=50000,
            max=500000,
            step=50000,
            description='Batch Size:',
            continuous_update=False,
            tooltip='Flows per ML batch',
            style={'description_width': 'initial'}
        )
        
        # ========== ANALYSIS OPTIONS ==========
        analysis_mode = widgets.RadioButtons(
            options=['flow', 'semantic', 'combined'],
            value='combined',
            description='Analysis Mode:',
            style={'description_width': 'initial'}
        )
        
        deep_inspection = widgets.Checkbox(
            value=True,
            description='Enable NLP Deep Inspection',
            tooltip='May increase processing time',
            indent=False
        )
        
        # ========== ML MODEL SELECTION ==========
        model_selector = widgets.SelectMultiple(
            options=['sgd', 'lightgbm', 'xgboost_incremental', 'random_forest'],
            value=['sgd', 'lightgbm'],
            description='ML Models:',
            tooltip='Memory-efficient models for large datasets',
            rows=4,
            style={'description_width': 'initial'},
            disabled=skip_ml.value
        )
        
        # Disable models when ML is skipped
        def toggle_models(change):
            model_selector.disabled = change['new']
        skip_ml.observe(toggle_models, names='value')
        
        # Progress output area
        output_area = widgets.Output()
        
        # ========== VALIDATE BUTTON ==========
        validate_btn = widgets.Button(
            description='Validate & Analyze Memory',
            button_style='success',
            layout=widgets.Layout(width='250px', height='40px')
        )
        
        def validate_config(b):
            """
            Callback function executed when validate button is clicked.
            Validates all inputs and sets configuration values.
            """
            with output_area:
                clear_output()
                
                # Get values from all widgets
                cls.PCAP_FILE = pcap_input.value.strip().strip('"')
                cls.OUTPUT_DIR = output_dir.value
                cls.MAX_MEMORY_GB = memory_slider.value
                cls.MAX_FLOWS_IN_MEMORY = max_flows.value
                cls.CHUNK_SIZE = chunk_slider.value
                cls.BATCH_SIZE = batch_slider.value
                cls.USE_SAMPLING = use_sampling.value
                cls.SAMPLE_SIZE = sample_size.value
                cls.ANALYSIS_MODE = analysis_mode.value
                cls.DEEP_INSPECTION = deep_inspection.value
                cls.SELECTED_MODELS = list(model_selector.value)
                cls.SKIP_ML = skip_ml.value
                cls.STORAGE_FORMAT = storage_format.value
                cls.USE_PARQUET = (storage_format.value == 'parquet')
                
                # Process CSV label files
                csv_lines = label_input.value.strip().split('\n')
                csv_files = [f.strip().strip('"') for f in csv_lines if f.strip()]
                
                # Validate PCAP exists
                if not os.path.exists(cls.PCAP_FILE):
                    print(f"Error: PCAP file not found: {cls.PCAP_FILE}")
                    return
                
                # Analyze memory requirements
                strategy = cls.check_memory_requirements()
                
                # Validate CSV files if provided
                if csv_files:
                    cls.USE_CICIDS_LABELS = True
                    valid_csvs = []
                    for csv in csv_files:
                        if os.path.exists(csv):
                            valid_csvs.append(csv)
                            print(f"Found: {os.path.basename(csv)}")
                        else:
                            print(f"Warning: Not found: {csv}")
                    
                    if valid_csvs:
                        cls.LABEL_FILES = valid_csvs
                
                # Create required directories
                os.makedirs(cls.OUTPUT_DIR, exist_ok=True)
                os.makedirs(cls.TEMP_DIR, exist_ok=True)
                
                # Display final configuration
                cls.display_config()
                
                print(f"\nProcessing Strategy: {strategy}")
                print(f"Storage Format: {cls.STORAGE_FORMAT.upper()}")
                if cls.SKIP_ML:
                    print("ML TRAINING WILL BE SKIPPED (Test Mode)")
                print("Ready to start analysis!")
        
        validate_btn.on_click(validate_config)
        
        # ========== LAYOUT ORGANIZATION ==========
        memory_box = widgets.VBox([
            widgets.HTML("<h4>Memory Settings (Optimized for 32GB)</h4>"),
            memory_slider,
            max_flows,
            use_sampling,
            sample_size
        ])
        
        file_box = widgets.VBox([
            widgets.HTML("<h4>Input/Output Files</h4>"),
            pcap_input,
            label_input,
            output_dir,
            storage_format
        ])
        
        processing_box = widgets.VBox([
            widgets.HTML("<h4>Processing Settings</h4>"),
            chunk_slider,
            batch_slider,
            analysis_mode,
            deep_inspection,
            skip_ml
        ])
        
        model_box = widgets.VBox([
            widgets.HTML("<h4>ML Models (Memory-Efficient)</h4>"),
            model_selector
        ])
        
        # Display complete interface
        display(widgets.VBox([
            memory_box,
            widgets.HTML("<hr>"),
            file_box,
            widgets.HTML("<hr>"),
            processing_box,
            widgets.HTML("<hr>"),
            model_box,
            widgets.HTML("<hr>"),
            validate_btn,
            output_area
        ]))
        
        return output_area
    
    @classmethod
    def display_config(cls):
        """Displays a summary of the current configuration"""
        print("\n" + "="*60)
        print("CONFIGURATION SUMMARY")
        print("="*60)
        print(f"PCAP: {os.path.basename(cls.PCAP_FILE)}")
        print(f"Output: {cls.OUTPUT_DIR}")
        print(f"Memory Limit: {cls.MAX_MEMORY_GB} GB")
        print(f"Processing Strategy: {'Disk-based' if cls.USE_DISK_CACHE else 'Memory-based'}")
        print(f"Storage Format: {cls.STORAGE_FORMAT}")
        print(f"Chunk Size: {cls.CHUNK_SIZE:,} packets")
        print(f"Batch Size: {cls.BATCH_SIZE:,} flows")
        print(f"CPU Threads: {cls.N_JOBS if cls.N_JOBS > 0 else 'All available'}")
        if cls.USE_SAMPLING:
            print(f"ML Sample Size: {cls.SAMPLE_SIZE:,} flows")
        if cls.SKIP_ML:
            print("ML Training: DISABLED (Test Mode)")
        print("="*60)

# Run the interactive UI
output = Config.setup_interactive_ui()

# ### Step 2.5: (OPTIONAL) Dataset Preparation (For Testing & Debugging) Use this to make smaller PCAP file and csv to save time. 

In [ ]:
# Cell 2.5: Test Dataset Preparation - Quick Validation Before Full Run
"""
PURPOSE: Create small test datasets to validate pipeline functionality
This cell helps you test the entire pipeline in minutes instead of hours.

WHAT THIS CELL DOES:
1. Creates a smaller PCAP file from your main file
2. Extracts corresponding CSV label rows  
3. Provides UI to customize test size
4. Validates pipeline works before committing to full analysis

WHY THIS MATTERS:
- Catches errors in 5 minutes instead of 4 hours
- Validates file locking fixes work
- Tests memory settings are appropriate
- Confirms CSV label matching functions
"""

import subprocess
import pandas as pd
import os
from IPython.display import display, HTML
import ipywidgets as widgets

class TestDatasetCreator:
    @staticmethod
    def create_test_ui():
        """
        Creates an interactive UI for generating test datasets
        """
        display(HTML("<h2>Test Dataset Generator</h2>"))
        display(HTML("<p>Create small test files to validate pipeline before full run</p>"))
        
        # Packet count slider
        packet_count = widgets.IntSlider(
            value=100000,
            min=10000,
            max=500000,
            step=10000,
            description='Test Packets:',
            style={'description_width': 'initial'},
            tooltip='Number of packets for test PCAP'
        )
        
        # CSV row count
        csv_rows = widgets.IntText(
            value=5000,
            description='CSV Rows:',
            style={'description_width': 'initial'},
            tooltip='Number of label rows to extract'
        )
        
        # Source file input
        source_pcap = widgets.Text(
            value=Config.PCAP_FILE if Config.PCAP_FILE else '',
            description='Source PCAP:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='70%')
        )
        
        # Source CSV input  
        source_csv = widgets.Textarea(
            value='\n'.join(Config.LABEL_FILES) if Config.LABEL_FILES else '',
            description='Source CSVs:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='70%', height='80px'),
            placeholder='Enter CSV paths (one per line)'
        )
        
        # Output directory
        test_dir = widgets.Text(
            value='./test_data',
            description='Test Directory:',
            style={'description_width': 'initial'}
        )
        
        # Progress output
        output_area = widgets.Output()
        
        # Create button
        create_btn = widgets.Button(
            description='Create Test Datasets',
            button_style='primary',
            layout=widgets.Layout(width='200px', height='40px')
        )
        
        def create_test_data(b):
            with output_area:
                clear_output()
                
                # Get values
                pcap_path = source_pcap.value.strip()
                csv_paths = [f.strip() for f in source_csv.value.strip().split('\n') if f.strip()]
                test_path = test_dir.value
                num_packets = packet_count.value
                num_rows = csv_rows.value
                
                # Validate inputs
                if not os.path.exists(pcap_path):
                    print(f" Error: PCAP file not found: {pcap_path}")
                    return
                
                # Create test directory
                os.makedirs(test_path, exist_ok=True)
                
                print("="*60)
                print("CREATING TEST DATASETS")
                print("="*60)
                
                # Create test PCAP
                test_pcap = os.path.join(test_path, f'test_{num_packets}_packets.pcap')
                print(f"\n1. Creating test PCAP with {num_packets:,} packets...")
                
                try:
                    # Use tcpdump to extract packets
                    cmd = f'tcpdump -r "{pcap_path}" -w "{test_pcap}" -c {num_packets}'
                    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
                    
                    if os.path.exists(test_pcap):
                        size_mb = os.path.getsize(test_pcap) / (1024*1024)
                        print(f"   ✓ Created: {test_pcap}")
                        print(f"   Size: {size_mb:.2f} MB")
                    else:
                        # Fallback to Python method if tcpdump fails
                        print("   tcpdump failed, using Python method...")
                        TestDatasetCreator.create_test_pcap_python(pcap_path, test_pcap, num_packets)
                        
                except Exception as e:
                    print(f"   Error creating PCAP: {e}")
                    print("   Trying Python-based extraction...")
                    TestDatasetCreator.create_test_pcap_python(pcap_path, test_pcap, num_packets)
                
                # Create test CSVs
                if csv_paths:
                    print(f"\n2. Creating test CSVs with {num_rows:,} rows each...")
                    test_csvs = []
                    
                    for csv_path in csv_paths:
                        if not os.path.exists(csv_path):
                            print(f"   CSV not found: {csv_path}")
                            continue
                        
                        # Read and subset CSV
                        csv_name = os.path.basename(csv_path)
                        test_csv = os.path.join(test_path, f'test_{csv_name}')
                        
                        try:
                            df = pd.read_csv(csv_path, encoding='latin-1', nrows=num_rows)
                            df.to_csv(test_csv, index=False)
                            test_csvs.append(test_csv)
                            
                            # Show label distribution
                            label_col = None
                            for col in ['Label', 'label', ' Label']:
                                if col in df.columns:
                                    label_col = col
                                    break
                            
                            if label_col:
                                print(f"\n   ✓ Created: {test_csv}")
                                print(f"   Label distribution:")
                                for label, count in df[label_col].value_counts().head(5).items():
                                    print(f"      {label}: {count}")
                        
                        except Exception as e:
                            print(f"   Error processing {csv_name}: {e}")
                else:
                    test_csvs = []
                    print("\n2. No CSV files provided, skipping label extraction")
                
                # Generate configuration code
                print("\n" + "="*60)
                print("TEST CONFIGURATION")
                print("="*60)
                print("\nAdd this to your Config or use directly:\n")
                print(f"Config.PCAP_FILE = r'{test_pcap}'")
                print(f"Config.LABEL_FILES = {test_csvs}")
                print(f"Config.MAX_PACKETS = 0  # Process all packets in test file")
                print(f"Config.SAMPLE_SIZE = {min(50000, num_packets // 2)}")
                
                # Estimate time
                est_time = num_packets / 20000  # ~20K packets per minute
                print(f"\nEstimated test time: {est_time:.1f} minutes")
                print("\n Test datasets created successfully!")
                print("   Run the pipeline with these files to validate before full analysis")
        
        create_btn.on_click(create_test_data)
        
        # Layout
        display(widgets.VBox([
            widgets.HTML("<h4>Test Size Configuration</h4>"),
            packet_count,
            csv_rows,
            widgets.HTML("<hr>"),
            widgets.HTML("<h4>Source Files</h4>"),
            source_pcap,
            source_csv,
            widgets.HTML("<hr>"),
            widgets.HTML("<h4>Output Location</h4>"),
            test_dir,
            widgets.HTML("<hr>"),
            create_btn,
            output_area
        ]))
    
    @staticmethod
    def create_test_pcap_python(source_pcap, output_pcap, packet_count):
        """
        Python fallback method to create test PCAP if tcpdump unavailable
        """
        from scapy.all import PcapReader, PcapWriter
        
        try:
            with PcapReader(source_pcap) as reader:
                with PcapWriter(output_pcap) as writer:
                    for i, packet in enumerate(reader):
                        if i >= packet_count:
                            break
                        writer.write(packet)
            
            if os.path.exists(output_pcap):
                size_mb = os.path.getsize(output_pcap) / (1024*1024)
                print(f"   ✓ Created using Python: {output_pcap}")
                print(f"   Size: {size_mb:.2f} MB")
        except Exception as e:
            print(f"   Error with Python method: {e}")

# Run the UI
TestDatasetCreator.create_test_ui()

# ### Step 3:  Flow Feature Extractor Implementation

In [4]:
# Cell 3: Flow Feature Extractor - Network Traffic Statistical Analysis
"""
PURPOSE: Extract statistical features from network flows
This class analyzes network packets and groups them into flows, then extracts
statistical features that help identify malicious traffic patterns.

WHAT THIS CELL DOES:
1. Groups packets into bidirectional flows (conversations between hosts)
2. Extracts per-packet features (size, flags, ports, timing)
3. Aggregates packet features into flow-level statistics
4. Implements CICFlowMeter-style feature extraction
5. Manages memory by flushing old flows to disk

KEY CONCEPTS:
- FLOW: A sequence of packets between two endpoints (identified by 5-tuple)
- 5-TUPLE: (src_ip, src_port, dst_ip, dst_port, protocol)
- BIDIRECTIONAL: Treats A→B and B→A as the same flow
- FLOW TIMEOUT: After 120 seconds of inactivity, flow is considered complete

FEATURES EXTRACTED:
1. Timing Features:
   - Flow duration
   - Inter-arrival times (IAT) statistics
   - Packets/bytes per second
   - Active/Idle time statistics (CICIDS-specific)

2. Size Features:
   - Packet length statistics (min, max, mean, std)
   - Total bytes/packets
   - Payload sizes
   - Forward/Backward packet statistics

3. TCP Flag Features:
   - SYN, ACK, FIN, RST, PSH, URG counts
   - Used to detect scanning, flooding attacks

4. Protocol Features:
   - TTL values
   - Port numbers
   - Protocol type

MEMORY OPTIMIZATION:
- Flushes flows to Parquet when memory limit reached
- Uses flow timeout to prevent infinite accumulation
- Stores features as float32 instead of float64
- Caps packets per flow to prevent memory bloat

IMPORTANT CHANGES:
- Fixed bidirectional flow identification bug
- Using Parquet instead of HDF5 (no file locking)
- Added CICIDS-specific features (Active/Idle, Fwd/Bwd separation)
- Optimized entropy calculation with numpy
- Added packet count cap per flow
"""

class MemoryOptimizedFlowExtractor:
    def __init__(self, max_flows_in_memory=50000, flow_timeout=120):
        """
        Initialize flow feature extractor with memory management.
        
        Args:
            max_flows_in_memory: Maximum concurrent flows before flushing to disk
            flow_timeout: Seconds of inactivity before flow is complete
        """
        self.flows = {}  # Dictionary to store active flows
        self.max_flows_in_memory = max_flows_in_memory
        self.flow_timeout = flow_timeout
        self.flow_counter = 0  # Total flows seen
        self.batch_counter = 0  # Number of batches written to disk
        
        # Create Parquet storage directory (replacing HDF5)
        self.output_dir = os.path.join(Config.TEMP_DIR, 'flow_features')
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Pre-compile optimizations
        self.setup_optimizations()
    
    def setup_optimizations(self):
        """
        Pre-compile and optimize frequently used operations.
        This improves performance significantly.
        """
        # Pre-calculate byte frequency table for entropy
        self.byte_frequencies = np.zeros(256, dtype=np.float32)
        
        # Maximum packets per flow to prevent memory issues
        self.max_packets_per_flow = Config.MAX_PACKETS_PER_FLOW
    
    def get_flow_id(self, packet):
        """
        Generate unique identifier for a network flow.
        FIXED: Properly handles bidirectional flow identification.
        
        Uses 5-tuple (IPs, ports, protocol) to identify flows.
        Makes flows bidirectional by sorting endpoints correctly.
        
        Returns:
            str: 16-character hash identifying the flow
        """
        try:
            if IP not in packet:
                return None
                
            src_ip = packet[IP].src
            dst_ip = packet[IP].dst
            proto = packet[IP].proto
            
            # Extract ports if TCP/UDP
            src_port = dst_port = 0
            if TCP in packet:
                src_port = packet[TCP].sport
                dst_port = packet[TCP].dport
            elif UDP in packet:
                src_port = packet[UDP].sport
                dst_port = packet[UDP].dport
            
            # FIXED: Proper bidirectional flow identification
            # Sort by IP first, then by port if IPs are equal
            if src_ip < dst_ip:
                flow_key = f"{src_ip}:{src_port}-{dst_ip}:{dst_port}-{proto}"
            elif src_ip > dst_ip:
                flow_key = f"{dst_ip}:{dst_port}-{src_ip}:{src_port}-{proto}"
            else:  # Same IP (rare but possible)
                if src_port <= dst_port:
                    flow_key = f"{src_ip}:{src_port}-{dst_ip}:{dst_port}-{proto}"
                else:
                    flow_key = f"{dst_ip}:{dst_port}-{src_ip}:{src_port}-{proto}"
            
            # Create hash for efficient lookup
            return hashlib.md5(flow_key.encode()).hexdigest()[:16]
        except:
            return None
    
    def extract_packet_features(self, packet):
        """
        Extract features from a single packet.
        These features will be aggregated into flow statistics.
        
        Features include:
        - Basic: timestamp, packet length, direction
        - IP: TTL, protocol, IPs (for direction determination)
        - TCP: flags, window size, ports
        - Payload: size and entropy (randomness)
        """
        features = {}
        
        try:
            # Basic packet features
            features['timestamp'] = float(packet.time)
            features['packet_length'] = len(packet)
            
            # IP layer features
            if IP in packet:
                features['ttl'] = packet[IP].ttl
                features['protocol'] = packet[IP].proto
                features['src_ip'] = packet[IP].src  # For direction determination
                features['dst_ip'] = packet[IP].dst
                
            # TCP layer features
            if TCP in packet:
                features['tcp_flags'] = int(packet[TCP].flags)
                features['window_size'] = packet[TCP].window
                features['src_port'] = packet[TCP].sport
                features['dst_port'] = packet[TCP].dport
                
                # Individual flag extraction (for detecting attacks)
                tcp_flags = packet[TCP].flags
                features['flag_syn'] = bool(tcp_flags & 2)   # SYN flag
                features['flag_ack'] = bool(tcp_flags & 16)  # ACK flag
                features['flag_fin'] = bool(tcp_flags & 1)   # FIN flag
                features['flag_rst'] = bool(tcp_flags & 4)   # RST flag
                features['flag_psh'] = bool(tcp_flags & 8)   # PSH flag
                features['flag_urg'] = bool(tcp_flags & 32)  # URG flag (CICIDS feature)
                
            # UDP layer features
            elif UDP in packet:
                features['src_port'] = packet[UDP].sport
                features['dst_port'] = packet[UDP].dport
                
            # Payload features (important for detecting malware)
            if Raw in packet:
                payload = bytes(packet[Raw])
                features['payload_size'] = len(payload)
                features['payload_entropy'] = self.fast_entropy(payload)
            else:
                features['payload_size'] = 0
                features['payload_entropy'] = 0
                
        except Exception as e:
            pass  # Skip problematic packets
            
        return features
    
    def fast_entropy(self, data):
        """
        OPTIMIZED: Calculate Shannon entropy using numpy for speed.
        High entropy suggests encryption/compression (possibly malware).
        Low entropy suggests plain text.
        
        Returns:
            float: Entropy value (0-8 bits)
        """
        if not data or len(data) == 0:
            return 0
        
        # Use numpy for fast calculation
        counts = np.bincount(np.frombuffer(data, dtype=np.uint8), minlength=256)
        probs = counts[counts > 0] / len(data)
        return -np.sum(probs * np.log2(probs))
    
    def flush_old_flows(self, current_time, force_all=False):
        """
        Save completed flows to disk using Parquet format.
        A flow is complete if it hasn't seen packets for flow_timeout seconds.
        
        IMPORTANT: Using Parquet instead of HDF5 for Windows reliability.
        
        Args:
            current_time: Timestamp of current packet
            force_all: Force flush all flows regardless of timeout
        """
        flows_to_flush = []
        
        # Identify flows that have timed out
        for flow_id, flow_data in self.flows.items():
            packets = flow_data['packets']
            if not packets:
                continue
                
            last_packet_time = packets[-1]['timestamp']
            time_since_last = current_time - last_packet_time
            
            if force_all or time_since_last > self.flow_timeout:
                # Aggregate packet features into flow features
                features = self.aggregate_flow_features(flow_data)
                if features:
                    features['flow_id'] = flow_id
                    flows_to_flush.append(features)
        
        # Save to disk if we have flows to flush
        if flows_to_flush:
            # Convert to DataFrame
            df_batch = pd.DataFrame(flows_to_flush)
            
            # Optimize memory usage with proper dtypes
            df_batch = self.optimize_dtypes(df_batch)
            
            # Save to Parquet (no file locking issues!)
            batch_file = os.path.join(
                self.output_dir, 
                f'batch_{self.batch_counter:04d}.parquet'
            )
            df_batch.to_parquet(
                batch_file,
                engine='pyarrow',
                compression='snappy',
                index=False
            )
            
            self.batch_counter += 1
            
            # Remove flushed flows from memory
            for features in flows_to_flush:
                del self.flows[features['flow_id']]
            
            print(f"  Flushed {len(flows_to_flush):,} flows to {batch_file}")
            
            # Force garbage collection to free memory
            gc.collect()
    
    def optimize_dtypes(self, df):
        """
        Optimize DataFrame dtypes for memory efficiency and storage.
        Specifically optimized for CICIDS features.
        """
        for col in df.columns:
            col_lower = col.lower()
            
            # Flag counts are small integers
            if 'flag' in col_lower or '_count' in col_lower:
                df[col] = df[col].astype(np.int8)
            # Ports fit in int16
            elif 'port' in col_lower:
                df[col] = df[col].astype(np.int16)
            # Packet/byte counts
            elif 'packet' in col_lower or 'byte' in col_lower:
                if 'rate' not in col_lower and '/s' not in col_lower:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.float32)
            # Float features
            elif df[col].dtype == np.float64:
                df[col] = df[col].astype(np.float32)
        
        return df
    
    def aggregate_flow_features(self, flow_data):
        """
        Aggregate packet-level features into flow-level statistics.
        This is where we calculate the actual features used for ML.
        
        ENHANCED: Now includes CICIDS-specific features like Active/Idle times
        and Forward/Backward packet separation.
        
        Creates statistical summaries that capture flow behavior:
        - Duration and timing patterns
        - Size distributions
        - Flag patterns
        - Rate calculations
        - Active/Idle statistics (CICIDS)
        - Forward/Backward statistics (CICIDS)
        """
        packets = flow_data['packets']
        if not packets:
            return None
            
        # Sort packets by timestamp
        packets = sorted(packets, key=lambda x: x.get('timestamp', 0))
        
        # Determine flow direction based on first packet
        first_packet = packets[0]
        flow_src_ip = first_packet.get('src_ip', '')
        
        # Separate forward and backward packets (CICIDS feature)
        fwd_packets = [p for p in packets if p.get('src_ip') == flow_src_ip]
        bwd_packets = [p for p in packets if p.get('src_ip') != flow_src_ip]
        
        # Extract timestamp array
        timestamps = [p['timestamp'] for p in packets]
        duration = max(timestamps) - min(timestamps) if len(timestamps) > 1 else 0
        
        # Calculate inter-arrival times (time between packets)
        iats = np.diff(timestamps) if len(timestamps) > 1 else [0]
        
        # Active/Idle time calculation (CICIDS-specific)
        active_times = []
        idle_times = []
        if len(iats) > 0:
            active_times = [t for t in iats if t < 1.0]  # Active if IAT < 1 second
            idle_times = [t for t in iats if t >= 1.0]    # Idle if IAT >= 1 second
        
        # Build feature dictionary
        features = {
            # ===== BASIC FLOW STATISTICS =====
            'flow_duration': duration,
            'total_packets': len(packets),
            'total_bytes': sum(p.get('packet_length', 0) for p in packets),
            
            # ===== FORWARD DIRECTION STATISTICS (CICIDS) =====
            'fwd_packets': len(fwd_packets),
            'fwd_bytes': sum(p.get('packet_length', 0) for p in fwd_packets),
            'fwd_packet_length_max': max([p.get('packet_length', 0) for p in fwd_packets]) if fwd_packets else 0,
            'fwd_packet_length_min': min([p.get('packet_length', 0) for p in fwd_packets]) if fwd_packets else 0,
            'fwd_packet_length_mean': np.mean([p.get('packet_length', 0) for p in fwd_packets]) if fwd_packets else 0,
            'fwd_packet_length_std': np.std([p.get('packet_length', 0) for p in fwd_packets]) if fwd_packets else 0,
            
            # ===== BACKWARD DIRECTION STATISTICS (CICIDS) =====
            'bwd_packets': len(bwd_packets),
            'bwd_bytes': sum(p.get('packet_length', 0) for p in bwd_packets),
            'bwd_packet_length_max': max([p.get('packet_length', 0) for p in bwd_packets]) if bwd_packets else 0,
            'bwd_packet_length_min': min([p.get('packet_length', 0) for p in bwd_packets]) if bwd_packets else 0,
            'bwd_packet_length_mean': np.mean([p.get('packet_length', 0) for p in bwd_packets]) if bwd_packets else 0,
            'bwd_packet_length_std': np.std([p.get('packet_length', 0) for p in bwd_packets]) if bwd_packets else 0,
            
            # ===== INTER-ARRIVAL TIME STATISTICS =====
            'flow_iat_mean': np.mean(iats) if len(iats) > 0 else 0,
            'flow_iat_std': np.std(iats) if len(iats) > 0 else 0,
            'flow_iat_max': max(iats) if len(iats) > 0 else 0,
            'flow_iat_min': min(iats) if len(iats) > 0 else 0,
            
            # ===== ACTIVE/IDLE TIME STATISTICS (CICIDS) =====
            'active_mean': np.mean(active_times) if active_times else 0,
            'active_std': np.std(active_times) if active_times else 0,
            'active_max': max(active_times) if active_times else 0,
            'active_min': min(active_times) if active_times else 0,
            'idle_mean': np.mean(idle_times) if idle_times else 0,
            'idle_std': np.std(idle_times) if idle_times else 0,
            'idle_max': max(idle_times) if idle_times else 0,
            'idle_min': min(idle_times) if idle_times else 0,
            
            # ===== FLOW RATE FEATURES =====
            'packets_per_second': len(packets) / duration if duration > 0 else 0,
            'bytes_per_second': sum(p.get('packet_length', 0) for p in packets) / duration if duration > 0 else 0,
            
            # ===== PROTOCOL FEATURES =====
            'avg_ttl': np.mean([p.get('ttl', 0) for p in packets]),
            'protocol': packets[0].get('protocol', 0) if packets else 0,
            
            # ===== TCP FLAG STATISTICS (Enhanced for CICIDS) =====
            'syn_count': sum(p.get('flag_syn', 0) for p in packets),
            'ack_count': sum(p.get('flag_ack', 0) for p in packets),
            'fin_count': sum(p.get('flag_fin', 0) for p in packets),
            'rst_count': sum(p.get('flag_rst', 0) for p in packets),
            'psh_count': sum(p.get('flag_psh', 0) for p in packets),
            'urg_count': sum(p.get('flag_urg', 0) for p in packets),  # Added URG flag
            
            # Forward/Backward flag counts (CICIDS)
            'fwd_psh_flags': sum(p.get('flag_psh', 0) for p in fwd_packets),
            'bwd_psh_flags': sum(p.get('flag_psh', 0) for p in bwd_packets),
            'fwd_urg_flags': sum(p.get('flag_urg', 0) for p in fwd_packets),
            'bwd_urg_flags': sum(p.get('flag_urg', 0) for p in bwd_packets),
            
            # ===== PAYLOAD STATISTICS =====
            'total_payload_bytes': sum(p.get('payload_size', 0) for p in packets),
            'avg_payload_size': np.mean([p.get('payload_size', 0) for p in packets]),
            'avg_entropy': np.mean([p.get('payload_entropy', 0) for p in packets]),
            
            # ===== PORT INFORMATION =====
            'src_port': packets[0].get('src_port', 0) if packets else 0,
            'dst_port': packets[0].get('dst_port', 0) if packets else 0,
        }
        
        # Convert to float32 for memory efficiency
        for key in features:
            if isinstance(features[key], (int, float)):
                features[key] = np.float32(features[key])
        
        return features
    
    def process_pcap(self, pcap_file, chunk_size=10000, max_packets=0):
        """
        Main processing function - reads PCAP and extracts flow features.
        Implements streaming processing to handle files larger than RAM.
        
        ENHANCED: Better memory management with packet count limits per flow.
        
        Processing steps:
        1. Read packets one by one
        2. Group into flows (with packet limit per flow)
        3. Extract features
        4. Flush to Parquet when memory limit reached
        5. Handle timeouts for inactive flows
        
        Returns:
            str: Path to directory containing Parquet files
        """
        print(f"\nProcessing PCAP with memory optimization: {pcap_file}")
        print(f"Max flows in memory: {self.max_flows_in_memory:,}")
        print(f"Max packets per flow: {self.max_packets_per_flow:,}")
        print(f"Flow timeout: {self.flow_timeout} seconds")
        print(f"Output format: Parquet (Windows-optimized)")
        
        packet_count = 0
        last_flush_time = None
        
        try:
            # Open PCAP file for streaming read
            with PcapReader(pcap_file) as pcap_reader:
                # Process packets one by one
                for packet in tqdm(pcap_reader, desc="Extracting flow features"):
                    packet_count += 1
                    
                    # Check packet limit
                    if max_packets > 0 and packet_count > max_packets:
                        break
                    
                    # Get flow identifier
                    flow_id = self.get_flow_id(packet)
                    if not flow_id:
                        continue
                    
                    # Extract packet features
                    packet_features = self.extract_packet_features(packet)
                    current_time = packet_features.get('timestamp', 0)
                    
                    # Initialize flow if new
                    if flow_id not in self.flows:
                        self.flows[flow_id] = {
                            'packets': [],
                            'stats': defaultdict(float)  # Running statistics
                        }
                        self.flow_counter += 1
                    
                    # Add packet to flow (with limit to prevent memory bloat)
                    if len(self.flows[flow_id]['packets']) < self.max_packets_per_flow:
                        self.flows[flow_id]['packets'].append(packet_features)
                    else:
                        # Update statistics without storing packet
                        self.update_flow_stats(self.flows[flow_id]['stats'], packet_features)
                    
                    # Check if memory limit reached
                    if len(self.flows) >= self.max_flows_in_memory:
                        print(f"\n  Memory limit reached at packet {packet_count:,}")
                        self.flush_old_flows(current_time)
                    
                    # Periodic timeout check
                    if packet_count % chunk_size == 0:
                        # Check for timed-out flows
                        if last_flush_time and (current_time - last_flush_time) > self.flow_timeout:
                            self.flush_old_flows(current_time)
                            last_flush_time = current_time
                        
                        # Monitor system memory
                        mem_percent = psutil.virtual_memory().percent
                        if mem_percent > 80:
                            print(f"\n  High memory usage ({mem_percent:.1f}%), flushing flows...")
                            self.flush_old_flows(current_time, force_all=True)
                        
                        gc.collect()
            
            # Flush all remaining flows
            print("\nFlushing remaining flows...")
            self.flush_old_flows(float('inf'), force_all=True)
            
            print(f"\nProcessed {packet_count:,} packets")
            print(f"Total flows: {self.flow_counter:,}")
            print(f"Features saved to: {self.output_dir}")
            
            return self.output_dir
            
        except Exception as e:
            print(f"Error processing PCAP: {e}")
            raise
    
    def update_flow_stats(self, stats, packet_features):
        """
        Update running statistics for a flow when packet limit is exceeded.
        This allows us to track flow statistics without storing all packets.
        """
        stats['packet_count'] += 1
        stats['byte_count'] += packet_features.get('packet_length', 0)
        stats['payload_total'] += packet_features.get('payload_size', 0)
        
        # Update flag counts
        for flag in ['syn', 'ack', 'fin', 'rst', 'psh', 'urg']:
            if packet_features.get(f'flag_{flag}', False):
                stats[f'{flag}_count'] += 1
    
    def load_features_iterator(self, batch_size=50000):
        """
        Generator to load features in batches from Parquet files.
        Allows processing results without loading all data into memory.
        
        Yields:
            DataFrame: Batch of flow features
        """
        import glob
        parquet_files = sorted(glob.glob(os.path.join(self.output_dir, 'batch_*.parquet')))
        
        accumulated = []
        accumulated_size = 0
        
        for file in parquet_files:
            batch_df = pd.read_parquet(file)
            accumulated.append(batch_df)
            accumulated_size += len(batch_df)
            
            if accumulated_size >= batch_size:
                combined_df = pd.concat(accumulated, ignore_index=True)
                yield combined_df
                accumulated = []
                accumulated_size = 0
        
        # Yield remaining data
        if accumulated:
            yield pd.concat(accumulated, ignore_index=True)

# ### Step4: Semantic Feature Extractor Implementation

In [5]:
# Cell 4: Semantic Feature Extractor - Deep Packet Inspection & NLP Analysis
"""
PURPOSE: Analyze packet payloads for malicious content using pattern matching and NLP
This class performs deep packet inspection to detect attack signatures in payload content.

WHAT THIS CELL DOES:
1. Examines packet payloads (actual data being transmitted)
2. Detects attack patterns (SQL injection, XSS, command injection)
3. Performs NLP analysis to find obfuscated attacks
4. Calculates entropy and encoding detection
5. Identifies suspicious URLs and domain names

KEY DETECTION CAPABILITIES:
1. SQL Injection: SELECT, UNION, DROP TABLE patterns
2. Cross-Site Scripting (XSS): <script>, javascript:, alert()
3. Command Injection: bash commands, system calls
4. Directory Traversal: ../, /etc/passwd
5. Encoding Detection: Base64, hex, URL encoding
6. Obfuscation: Unusual character patterns, high entropy

WHY SEMANTIC ANALYSIS MATTERS:
- Flow features only see traffic patterns, not content
- Many attacks hide in seemingly normal traffic
- Attackers use encoding/obfuscation to evade detection
- NLP helps detect variations of known attacks

MEMORY OPTIMIZATION:
- Processes payloads in streaming fashion
- Limits payload analysis to first 1000 characters
- Flushes results to Parquet periodically
- Uses simplified NLP for speed
- Pre-compiles regex patterns

IMPORTANT CHANGES:
- Using Parquet instead of HDF5 for storage
- Pre-compiled regex patterns for better performance
- Optimized entropy calculation with numpy
- Added more comprehensive attack patterns
"""

class MemoryOptimizedSemanticExtractor:
    def __init__(self, max_flows_in_memory=50000):
        """
        Initialize semantic analyzer with pattern databases and NLP components.
        
        Args:
            max_flows_in_memory: Maximum flows before flushing to disk
        """
        self.max_flows_in_memory = max_flows_in_memory
        self.semantic_data = {}  # Stores semantic features per flow
        self.batch_counter = 0
        
        # Create Parquet storage directory
        self.output_dir = os.path.join(Config.TEMP_DIR, 'semantic_features')
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Pre-compile all regex patterns for performance
        self.compile_attack_patterns()
        
        # Initialize NLP analyzer (simplified version for memory efficiency)
        self.nlp_analyzer = SimplifiedNLPAnalyzer()
    
    def compile_attack_patterns(self):
        """
        Pre-compile regex patterns for better performance.
        Compiled patterns are much faster than compiling on each use.
        """
        # ===== SQL INJECTION PATTERNS =====
        sql_patterns = [
            r'SELECT.*FROM',      # Basic SELECT query
            r'INSERT.*INTO',      # INSERT injection
            r'UPDATE.*SET',       # UPDATE injection
            r'DELETE.*FROM',      # DELETE injection
            r'DROP.*TABLE',       # Table dropping
            r'UNION.*SELECT',     # UNION-based injection
            r'OR\s+1\s*=\s*1',   # Classic bypass: OR 1=1
            r'--\s*$',           # SQL comment injection
            r';\s*EXEC',         # Command execution
            r'xp_cmdshell',      # SQL Server command execution
            r'WAITFOR\s+DELAY',  # Time-based blind SQL
            r'BENCHMARK\s*\(',   # MySQL benchmark attack
        ]
        self.sql_patterns = [re.compile(p, re.IGNORECASE) for p in sql_patterns]
        
        # ===== COMMAND INJECTION PATTERNS =====
        cmd_patterns = [
            r';\s*ls\s+',        # List directory (Linux)
            r';\s*cat\s+',       # Read file (Linux)
            r';\s*wget\s+',      # Download file
            r';\s*curl\s+',      # HTTP request tool
            r';\s*nc\s+',        # Netcat (backdoor tool)
            r'/etc/passwd',      # Common target file
            r'/etc/shadow',      # Password hashes
            r'cmd\.exe',         # Windows command prompt
            r'powershell',       # Windows PowerShell
            r'bash\s+-c',        # Bash command execution
            r'sh\s+-c',          # Shell command execution
            r'eval\s*\(',        # Code evaluation
            r'exec\s*\(',        # Code execution
            r'system\s*\(',      # System call
        ]
        self.cmd_patterns = [re.compile(p, re.IGNORECASE) for p in cmd_patterns]
        
        # ===== XSS/SCRIPT INJECTION PATTERNS =====
        script_patterns = [
            r'<script',          # Script tag injection
            r'javascript:',      # JavaScript protocol
            r'onerror\s*=',     # Event handler injection
            r'onclick\s*=',     # Click event injection
            r'onload\s*=',      # Load event injection
            r'alert\s*\(',      # JavaScript alert
            r'document\.cookie', # Cookie theft
            r'document\.location', # Redirection
            r'eval\s*\(',       # Code evaluation
            r'<iframe',         # IFrame injection
            r'<embed',          # Embed tag injection
            r'<object',         # Object tag injection
            r'<img.*src.*=',    # Image injection
        ]
        self.script_patterns = [re.compile(p, re.IGNORECASE) for p in script_patterns]
        
        # ===== DIRECTORY TRAVERSAL PATTERNS =====
        traversal_patterns = [
            r'\.\./\.\./\.\./', # Multiple traversals
            r'\.\.\\\.\.\\\.\.\\'  # Windows traversals
        ]
        self.traversal_patterns = [re.compile(p) for p in traversal_patterns]
    
    def flush_semantic_features(self):
        """
        Save accumulated semantic features to Parquet and free memory.
        Called when memory limit is reached.
        
        Using Parquet instead of HDF5 for Windows reliability.
        """
        if not self.semantic_data:
            return
        
        # Convert dictionary to DataFrame
        df_batch = pd.DataFrame.from_dict(self.semantic_data, orient='index')
        df_batch['flow_id'] = df_batch.index
        df_batch = df_batch.reset_index(drop=True)
        
        # Optimize data types
        for col in df_batch.select_dtypes(include=[np.float64]).columns:
            df_batch[col] = df_batch[col].astype(np.float32)
        
        # Save to Parquet
        batch_file = os.path.join(
            self.output_dir,
            f'batch_{self.batch_counter:04d}.parquet'
        )
        df_batch.to_parquet(
            batch_file,
            engine='pyarrow',
            compression='snappy',
            index=False
        )
        
        self.batch_counter += 1
        print(f"  Flushed {len(df_batch):,} semantic features to {batch_file}")
        
        # Clear memory
        self.semantic_data.clear()
        gc.collect()
    
    def extract_semantic_features(self, packet):
        """
        Extract semantic features from packet payload.
        Analyzes actual data content for attack signatures.
        
        OPTIMIZED: Uses pre-compiled patterns and limits payload analysis.
        
        Returns:
            dict: Semantic feature values
        """
        features = {
            # ===== PROTOCOL INDICATORS =====
            'has_http': 0,       # HTTP traffic
            'has_dns': 0,        # DNS queries
            'has_smtp': 0,       # Email traffic
            'has_tls': 0,        # TLS/SSL traffic
            
            # ===== ATTACK INDICATORS =====
            'has_sql': 0,        # SQL injection detected
            'has_cmd': 0,        # Command injection detected
            'has_script': 0,     # Script injection detected
            'has_traversal': 0,  # Directory traversal detected
            
            # ===== SCORING =====
            'suspicious_score': 0,  # Overall suspicion level
            'content_length': 0,    # Payload size
            'attack_pattern_count': 0,  # Total patterns matched
            
            # ===== NLP FEATURES =====
            'nlp_malicious_confidence': 0,  # NLP-based threat score
            'nlp_pattern_score': 0,         # Pattern matching score
            'nlp_entropy_score': 0,         # Randomness score
            'nlp_encoding_detected': 0      # Encoding/obfuscation detected
        }
        
        try:
            # Check for HTTP
            if packet.haslayer('HTTP'):
                features['has_http'] = 1
                
                # Extract HTTP-specific features
                if hasattr(packet['HTTP'], 'Method'):
                    # POST/PUT methods often carry attack payloads
                    method = packet['HTTP'].Method
                    if method in [b'POST', b'PUT', b'PATCH']:
                        features['suspicious_score'] += 1
                    
                    # Check for suspicious headers
                    if hasattr(packet['HTTP'], 'Host'):
                        host = str(packet['HTTP'].Host)
                        # Check for IP addresses in Host header (suspicious)
                        if re.match(r'\d+\.\d+\.\d+\.\d+', host):
                            features['suspicious_score'] += 2
            
            # Check for DNS
            if packet.haslayer('DNS'):
                features['has_dns'] = 1
                # DNS tunneling detection
                if hasattr(packet['DNS'], 'qd') and packet['DNS'].qd:
                    query_name = str(packet['DNS'].qd.qname)
                    # Long DNS names might indicate tunneling
                    if len(query_name) > 50:
                        features['suspicious_score'] += 2
            
            # Check for TLS/SSL
            if packet.haslayer('TLS'):
                features['has_tls'] = 1
            
            # Extract and analyze payload
            if packet.haslayer('Raw'):
                payload = str(packet['Raw'].load)
                features['content_length'] = len(payload)
                
                # Limit payload analysis for performance (first 1000 chars)
                payload_sample = payload[:1000]
                
                # ===== PATTERN DETECTION WITH PRE-COMPILED PATTERNS =====
                # Check for SQL injection (check top 5 patterns for speed)
                for pattern in self.sql_patterns[:5]:
                    if pattern.search(payload_sample):
                        features['has_sql'] = 1
                        features['suspicious_score'] += 3
                        features['attack_pattern_count'] += 1
                        break
                
                # Check for command injection
                for pattern in self.cmd_patterns[:5]:
                    if pattern.search(payload_sample):
                        features['has_cmd'] = 1
                        features['suspicious_score'] += 5  # Higher score for OS commands
                        features['attack_pattern_count'] += 1
                        break
                
                # Check for script injection
                for pattern in self.script_patterns[:5]:
                    if pattern.search(payload_sample):
                        features['has_script'] = 1
                        features['suspicious_score'] += 2
                        features['attack_pattern_count'] += 1
                        break
                
                # Check for directory traversal
                if '../' in payload or '..\\' in payload:
                    features['has_traversal'] = 1
                    features['suspicious_score'] += 2
                    features['attack_pattern_count'] += 1
                
                # ===== NLP ANALYSIS =====
                # Perform deeper analysis if enabled
                if Config.DEEP_INSPECTION and len(payload) > 10:
                    nlp_features = self.nlp_analyzer.quick_analyze(payload_sample)
                    features.update(nlp_features)
                    
        except Exception as e:
            pass  # Skip problematic packets
        
        return features
    
    def process_pcap_streaming(self, pcap_file, chunk_size=10000, max_packets=0):
        """
        Process PCAP file for semantic analysis with streaming.
        Analyzes packet payloads for malicious content.
        
        Uses Parquet for storage instead of HDF5.
        
        Returns:
            str: Path to directory with Parquet files containing semantic features
        """
        print(f"\nSemantic analysis (memory-optimized): {pcap_file}")
        print(f"Output format: Parquet (Windows-optimized)")
        
        packet_count = 0
        flow_extractor = MemoryOptimizedFlowExtractor()  # Reuse flow ID logic
        
        try:
            with PcapReader(pcap_file) as pcap_reader:
                for packet in tqdm(pcap_reader, desc="Extracting semantic features"):
                    packet_count += 1
                    
                    if max_packets > 0 and packet_count > max_packets:
                        break
                    
                    # Get flow ID to group semantic features
                    flow_id = flow_extractor.get_flow_id(packet)
                    if not flow_id:
                        continue
                    
                    # Extract semantic features from payload
                    features = self.extract_semantic_features(packet)
                    
                    # Aggregate features by flow
                    if flow_id not in self.semantic_data:
                        self.semantic_data[flow_id] = defaultdict(float)
                    
                    # Sum up features for the flow
                    for key, value in features.items():
                        self.semantic_data[flow_id][key] += value
                    
                    # Check memory limit
                    if len(self.semantic_data) >= self.max_flows_in_memory:
                        self.flush_semantic_features()
                    
                    # Periodic memory check
                    if packet_count % chunk_size == 0:
                        mem_percent = psutil.virtual_memory().percent
                        if mem_percent > 80:
                            print(f"\n  High memory ({mem_percent:.1f}%), flushing...")
                            self.flush_semantic_features()
                        gc.collect()
            
            # Final flush
            self.flush_semantic_features()
            
            print(f"Processed {packet_count:,} packets")
            print(f"Semantic features saved to: {self.output_dir}")
            
            return self.output_dir
            
        except Exception as e:
            print(f"Error in semantic processing: {e}")
            raise


class SimplifiedNLPAnalyzer:
    """
    Lightweight NLP analyzer for payload inspection.
    Optimized for speed and memory efficiency.
    
    DETECTION METHODS:
    1. Keyword density analysis
    2. Entropy calculation (randomness)
    3. Encoding detection
    4. Character distribution analysis
    
    OPTIMIZATIONS:
    - Pre-compiled patterns
    - Numpy-based calculations
    - Limited analysis scope
    """
    
    def __init__(self):
        # Reduced keyword sets for memory efficiency
        self.sql_keywords = {'select', 'union', 'insert', 'drop', 'exec', 'declare', 'cast', 'convert', 'table', 'database'}
        self.xss_keywords = {'script', 'javascript', 'alert', 'onerror', 'onclick', 'document', 'cookie', 'eval', 'iframe'}
        self.cmd_keywords = {'bash', 'cmd', 'wget', 'curl', 'nc', 'telnet', 'ssh', 'powershell', 'exec', 'system'}
        
        # Pre-compiled encoding patterns for performance
        self.encoding_patterns = {
            'base64': re.compile(r'^[A-Za-z0-9+/]+=*$'),
            'hex': re.compile(r'^[0-9A-Fa-f]+$'),
            'url': re.compile(r'%[0-9A-Fa-f]{2}')
        }
    
    def quick_analyze(self, payload):
        """
        Perform quick NLP analysis on payload.
        Focuses on key indicators of malicious content.
        
        OPTIMIZED: Uses set operations and numpy for speed.
        
        Returns:
            dict: NLP feature scores
        """
        features = {}
        
        # Limit analysis to first 1000 characters for speed
        payload_sample = payload[:1000].lower()
        
        # ===== KEYWORD ANALYSIS (Optimized with sets) =====
        # Tokenize efficiently
        words = set(re.findall(r'\b\w+\b', payload_sample))
        
        # Count suspicious keywords using set intersection
        sql_score = len(words & self.sql_keywords)
        xss_score = len(words & self.xss_keywords)
        cmd_score = len(words & self.cmd_keywords)
        
        # Normalize scores (0-1 range)
        features['nlp_pattern_score'] = min((sql_score + xss_score + cmd_score) / 10, 1.0)
        
        # ===== ENTROPY ANALYSIS (Using numpy) =====
        # High entropy suggests encryption/obfuscation
        if len(payload) > 0:
            # Fast entropy calculation
            byte_counts = np.bincount(np.frombuffer(payload[:100].encode('utf-8', errors='ignore'), dtype=np.uint8), minlength=256)
            byte_probs = byte_counts[byte_counts > 0] / min(len(payload), 100)
            features['nlp_entropy_score'] = -np.sum(byte_probs * np.log2(byte_probs)) / 8  # Normalize to 0-1
        else:
            features['nlp_entropy_score'] = 0
        
        # ===== ENCODING DETECTION =====
        # Check for common encoding schemes
        encoding_detected = 0
        sample = payload_sample[:50]
        
        for pattern_name, pattern in self.encoding_patterns.items():
            if pattern.search(sample):
                encoding_detected = 1
                break
        
        features['nlp_encoding_detected'] = encoding_detected
        
        # ===== CHARACTER DISTRIBUTION ANALYSIS =====
        # Suspicious if too many special characters
        if len(payload_sample) > 0:
            special_chars = sum(1 for c in payload_sample if not c.isalnum() and not c.isspace())
            special_ratio = special_chars / len(payload_sample)
            features['nlp_special_char_ratio'] = min(special_ratio, 1.0)
        else:
            features['nlp_special_char_ratio'] = 0
        
        # ===== OVERALL MALICIOUS CONFIDENCE =====
        # Weighted combination of all indicators
        features['nlp_malicious_confidence'] = min(
            features['nlp_pattern_score'] * 0.4 +      # Pattern matching weight
            features['nlp_entropy_score'] * 0.2 +      # Entropy weight
            features['nlp_encoding_detected'] * 0.2 +  # Encoding weight
            features['nlp_special_char_ratio'] * 0.2,  # Special char weight
            1.0
        )
        
        return features

# ### Step 5: Combined Feature Extraction Pipeline

In [6]:
# Cell 5: Combined Feature Pipeline - Merging Flow & Semantic Features
"""
PURPOSE: Orchestrate feature extraction and merge different feature types
This class combines flow statistics with semantic analysis to create a comprehensive
feature set for machine learning.

WHAT THIS CELL DOES:
1. Coordinates flow and semantic feature extraction
2. Merges features from multiple sources using flow_id
3. Performs feature engineering (creates new features from existing ones)
4. Handles the entire extraction pipeline end-to-end
5. Manages disk-based merging for large datasets

KEY CONCEPTS:
- FEATURE FUSION: Combining statistical and content-based features
- FEATURE ENGINEERING: Creating derived features that better capture patterns
- DISK-BASED MERGE: Joining large datasets without loading into memory

FEATURE TYPES COMBINED:
1. Flow Features (from Cell 3):
   - Timing statistics
   - Packet sizes
   - TCP flags
   - Flow rates
   - Forward/Backward statistics (CICIDS)

2. Semantic Features (from Cell 4):
   - Attack pattern detection
   - NLP analysis scores
   - Entropy measurements
   - Protocol indicators

3. Engineered Features (created here):
   - Packet rate (packets/duration)
   - Average packet size
   - Port categories (well-known, registered)
   - Flag ratios (flags/total packets)

WHY COMBINE FEATURES:
- Flow features detect behavioral anomalies
- Semantic features detect content anomalies
- Combined view provides better attack detection
- Some attacks only visible through combination

IMPORTANT CHANGES:
- Using Parquet instead of HDF5 for storage
- Improved merging strategy for Windows
- Added CICIDS-specific engineered features
- Parallel processing support for feature engineering
"""

class MemoryOptimizedFeaturePipeline:
    def __init__(self):
        """
        Initialize the combined feature pipeline with both extractors.
        """
        # Initialize component extractors
        self.flow_extractor = MemoryOptimizedFlowExtractor(
            max_flows_in_memory=Config.MAX_FLOWS_IN_MEMORY,
            flow_timeout=Config.FLOW_TIMEOUT
        )
        self.semantic_extractor = MemoryOptimizedSemanticExtractor(
            max_flows_in_memory=Config.MAX_FLOWS_IN_MEMORY
        )
        
        # Output directory for combined features
        self.output_dir = os.path.join(Config.TEMP_DIR, 'combined_features')
        os.makedirs(self.output_dir, exist_ok=True)
    
    def extract_all_features(self, pcap_file, mode='combined'):
        """
        Main orchestration function - manages entire feature extraction process.
        
        Args:
            pcap_file: Path to PCAP file
            mode: 'flow' (statistics only), 'semantic' (content only), or 'combined' (both)
        
        Returns:
            str: Path to directory containing all features
        """
        feature_dirs = []  # List to track generated feature directories
        
        # ===== PHASE 1: FLOW FEATURE EXTRACTION =====
        if mode in ['flow', 'combined']:
            print("\n" + "="*60)
            print("PHASE 1: MEMORY-OPTIMIZED FLOW EXTRACTION")
            print("="*60)
            print("Extracting statistical features from network flows...")
            print("This analyzes packet timing, sizes, and patterns")
            print("Including CICIDS-specific features (Active/Idle, Fwd/Bwd)")
            
            flow_dir = self.flow_extractor.process_pcap(
                pcap_file,
                chunk_size=Config.CHUNK_SIZE,
                max_packets=Config.MAX_PACKETS
            )
            feature_dirs.append(('flow', flow_dir))
            print(f"✓ Flow features extracted to: {flow_dir}")
        
        # ===== PHASE 2: SEMANTIC FEATURE EXTRACTION =====
        if mode in ['semantic', 'combined'] and Config.DEEP_INSPECTION:
            print("\n" + "="*60)
            print("PHASE 2: MEMORY-OPTIMIZED SEMANTIC EXTRACTION")
            print("="*60)
            print("Analyzing packet payloads for malicious content...")
            print("This performs deep packet inspection and NLP analysis")
            
            semantic_dir = self.semantic_extractor.process_pcap_streaming(
                pcap_file,
                chunk_size=Config.CHUNK_SIZE,
                max_packets=Config.MAX_PACKETS
            )
            feature_dirs.append(('semantic', semantic_dir))
            print(f"✓ Semantic features extracted to: {semantic_dir}")
        
        # ===== PHASE 3: FEATURE MERGING =====
        if len(feature_dirs) > 1:
            print("\n" + "="*60)
            print("PHASE 3: MERGING FEATURES (DISK-BASED)")
            print("="*60)
            print("Combining flow and semantic features...")
            return self.merge_features_on_disk(feature_dirs)
        elif feature_dirs:
            return feature_dirs[0][1]  # Return single feature directory if only one type
        else:
            return None
    
    def merge_features_on_disk(self, feature_dirs):
        """
        Merge features from multiple Parquet directories without loading all into memory.
        Uses flow_id as the join key to combine features.
        
        OPTIMIZED: Uses Parquet's columnar format for efficient merging.
        
        Process:
        1. Read flow features in batches
        2. Find matching semantic features for each batch
        3. Merge on flow_id
        4. Apply feature engineering
        5. Save merged batch to new Parquet file
        
        Returns:
            str: Path to merged feature directory
        """
        print("Merging features using disk-based operations...")
        print("This preserves memory by processing in batches")
        
        # Extract directory paths
        flow_dir = feature_dirs[0][1]  # Flow features directory
        semantic_dir = feature_dirs[1][1] if len(feature_dirs) > 1 else None
        
        # Get all Parquet files
        import glob
        flow_files = sorted(glob.glob(os.path.join(flow_dir, 'batch_*.parquet')))
        
        batch_counter = 0
        
        for flow_file in tqdm(flow_files, desc="Merging batches"):
            # Load batch of flow features
            flow_batch = pd.read_parquet(flow_file)
            
            # Merge with semantic features if available
            if semantic_dir:
                # Get flow IDs from this batch
                flow_ids = set(flow_batch['flow_id'].values)
                
                # Load matching semantic features
                semantic_batch = self.load_matching_semantic_features(
                    semantic_dir, flow_ids
                )
                
                # Merge on flow_id (left join to keep all flows)
                if semantic_batch is not None and not semantic_batch.empty:
                    combined_batch = pd.merge(
                        flow_batch, semantic_batch,
                        on='flow_id', how='left'
                    )
                    
                    # Fill missing semantic features with zeros
                    semantic_cols = semantic_batch.columns.difference(['flow_id'])
                    combined_batch[semantic_cols] = combined_batch[semantic_cols].fillna(0)
                else:
                    combined_batch = flow_batch
            else:
                combined_batch = flow_batch
            
            # Apply feature engineering to create derived features
            combined_batch = self.engineer_features(combined_batch)
            
            # Optimize data types
            combined_batch = self.optimize_dtypes(combined_batch)
            
            # Save merged batch
            output_file = os.path.join(
                self.output_dir,
                f'batch_{batch_counter:04d}.parquet'
            )
            combined_batch.to_parquet(
                output_file,
                engine='pyarrow',
                compression='snappy',
                index=False
            )
            
            batch_counter += 1
            
            # Clean up memory
            del combined_batch
            gc.collect()
        
        print(f"✓ Features merged and saved to: {self.output_dir}")
        print(f"  Total batches processed: {batch_counter}")
        return self.output_dir
    
    def load_matching_semantic_features(self, semantic_dir, flow_ids):
        """
        Load semantic features that match given flow IDs.
        Efficient loading - only reads matching records.
        
        Args:
            semantic_dir: Directory containing semantic feature Parquet files
            flow_ids: Set of flow IDs to match
            
        Returns:
            DataFrame: Semantic features for matching flows
        """
        matching_features = []
        
        import glob
        semantic_files = sorted(glob.glob(os.path.join(semantic_dir, 'batch_*.parquet')))
        
        for file in semantic_files:
            batch = pd.read_parquet(file)
            
            # Filter for matching flow IDs
            matches = batch[batch['flow_id'].isin(flow_ids)]
            
            if not matches.empty:
                matching_features.append(matches)
        
        # Combine all matching features
        if matching_features:
            return pd.concat(matching_features, ignore_index=True)
        return pd.DataFrame()
    
    def engineer_features(self, df):
        """
        Create derived features that better capture attack patterns.
        Feature engineering is crucial for ML model performance.
        
        ENHANCED: Added CICIDS-specific engineered features.
        
        Engineered features include:
        1. Rate features: packets/second, bytes/second
        2. Ratio features: flag counts / total packets
        3. Port categories: well-known (<1024), registered (1024-49151)
        4. Suspicious indicators: binary flags for quick filtering
        5. CICIDS ratios: forward/backward packet ratios
        
        Args:
            df: DataFrame with raw features
            
        Returns:
            DataFrame: With additional engineered features
        """
        # ===== RATE FEATURES =====
        # Packet rate (packets per second)
        if 'total_packets' in df.columns and 'flow_duration' in df.columns:
            df['packet_rate'] = df['total_packets'] / (df['flow_duration'] + 0.001)  # Avoid division by zero
            df['packet_rate'] = df['packet_rate'].astype(np.float32)
        
        # Average packet size
        if 'total_bytes' in df.columns and 'total_packets' in df.columns:
            df['avg_packet_size'] = df['total_bytes'] / (df['total_packets'] + 1)
            df['avg_packet_size'] = df['avg_packet_size'].astype(np.float32)
        
        # ===== CICIDS-SPECIFIC RATIOS =====
        # Forward/Backward packet ratio
        if 'fwd_packets' in df.columns and 'bwd_packets' in df.columns:
            df['fwd_bwd_packets_ratio'] = df['fwd_packets'] / (df['bwd_packets'] + 1)
            df['fwd_bwd_packets_ratio'] = df['fwd_bwd_packets_ratio'].astype(np.float32)
        
        # Forward/Backward bytes ratio
        if 'fwd_bytes' in df.columns and 'bwd_bytes' in df.columns:
            df['fwd_bwd_bytes_ratio'] = df['fwd_bytes'] / (df['bwd_bytes'] + 1)
            df['fwd_bwd_bytes_ratio'] = df['fwd_bwd_bytes_ratio'].astype(np.float32)
        
        # ===== FLAG RATIO FEATURES =====
        # Calculate flag ratios (important for detecting SYN floods, etc.)
        flag_cols = ['syn_count', 'ack_count', 'fin_count', 'rst_count', 'psh_count', 'urg_count']
        if all(col in df.columns for col in flag_cols) and 'total_packets' in df.columns:
            for flag in flag_cols:
                ratio_name = f'{flag}_ratio'
                df[ratio_name] = df[flag] / (df['total_packets'] + 1)
                df[ratio_name] = df[ratio_name].astype(np.float32)
        
        # ===== PORT CATEGORY FEATURES =====
        # Categorize ports for better pattern recognition
        if 'dst_port' in df.columns:
            # Well-known ports (0-1023) - usually system services
            df['is_well_known_port'] = (df['dst_port'] < 1024).astype(np.int8)
            
            # Registered ports (1024-49151) - usually applications
            df['is_registered_port'] = ((df['dst_port'] >= 1024) & 
                                        (df['dst_port'] < 49152)).astype(np.int8)
            
            # Dynamic/private ports (49152-65535) - usually client connections
            df['is_dynamic_port'] = (df['dst_port'] >= 49152).astype(np.int8)
            
            # Common attack target ports
            attack_ports = {80, 443, 22, 23, 3389, 445, 139, 21, 25, 3306}
            df['is_common_target_port'] = df['dst_port'].isin(attack_ports).astype(np.int8)
        
        # ===== SUSPICIOUS INDICATORS =====
        # Binary flags for quick filtering
        if 'suspicious_score' in df.columns:
            df['is_suspicious'] = (df['suspicious_score'] > 0).astype(np.int8)
            df['highly_suspicious'] = (df['suspicious_score'] > 5).astype(np.int8)
        
        # ===== ATTACK COMBINATION FEATURES =====
        # Some attacks use specific combinations
        if 'has_sql' in df.columns and 'has_script' in df.columns:
            # SQL + Script often indicates complex web attack
            df['sql_and_script'] = ((df.get('has_sql', 0) > 0) & 
                                    (df.get('has_script', 0) > 0)).astype(np.int8)
        
        if 'has_cmd' in df.columns and 'nlp_encoding_detected' in df.columns:
            # Command injection + encoding often indicates obfuscated attack
            df['encoded_cmd'] = ((df.get('has_cmd', 0) > 0) & 
                                 (df.get('nlp_encoding_detected', 0) > 0)).astype(np.int8)
        
        # ===== PAYLOAD RATIO FEATURES =====
        if 'total_payload_bytes' in df.columns and 'total_bytes' in df.columns:
            # Ratio of payload to total traffic
            df['payload_ratio'] = df['total_payload_bytes'] / (df['total_bytes'] + 1)
            df['payload_ratio'] = df['payload_ratio'].astype(np.float32)
        
        # ===== ACTIVE/IDLE FEATURES (CICIDS) =====
        if 'active_mean' in df.columns and 'idle_mean' in df.columns:
            # Active to idle ratio
            df['active_idle_ratio'] = df['active_mean'] / (df['idle_mean'] + 0.001)
            df['active_idle_ratio'] = df['active_idle_ratio'].astype(np.float32)
        
        return df
    
    def optimize_dtypes(self, df):
        """
        Optimize DataFrame dtypes for memory efficiency.
        Reduces memory usage significantly.
        """
        for col in df.columns:
            col_type = df[col].dtype
            
            if col_type != 'object':
                c_min = df[col].min()
                c_max = df[col].max()
                
                # Integer optimization
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                
                # Float optimization
                elif str(col_type)[:5] == 'float':
                    df[col] = df[col].astype(np.float32)
        
        return df

# ### Step 6: CICIDS2017 Label Loader

In [7]:
# Cell 6: CICIDS Label Matcher - Ground Truth Integration
"""
PURPOSE: Match extracted flows with CICIDS2017 ground truth labels
This class integrates the official attack labels from CICIDS2017 dataset
to enable supervised learning.

WHAT THIS CELL DOES:
1. Loads CICIDS2017 CSV label files incrementally
2. Creates a mapping between network flows and attack types
3. Handles multiple CSV files from different days
4. Assigns numeric labels for ML training
5. Maintains attack type distribution statistics

CICIDS2017 ATTACK TYPES:
The dataset contains 15 different attack categories:
0. BENIGN - Normal, non-malicious traffic
1. Bot - Botnet traffic
2. DDoS - Distributed Denial of Service
3. DoS GoldenEye - Application layer DoS
4. DoS Hulk - Volume-based DoS
5. DoS Slowhttptest - Slow HTTP attack
6. DoS slowloris - Connection exhaustion
7. FTP-Patator - FTP brute force
8. Heartbleed - SSL vulnerability exploit
9. Infiltration - Network infiltration
10. PortScan - Port scanning activity
11. SSH-Patator - SSH brute force
12. Web Attack - Brute Force
13. Web Attack - SQL Injection
14. Web Attack - XSS

MATCHING STRATEGY:
- Primary: Match by 5-tuple (IPs, ports, protocol)
- Fallback: Match by destination port (majority vote)
- Default: Label as BENIGN if no match found

MEMORY OPTIMIZATION:
- Loads CSV files in chunks (100K rows at a time)
- Builds port-label cache instead of full flow mapping
- Processes labels incrementally without loading all
- Uses Parquet for output

IMPORTANT CHANGES:
- Using Parquet instead of HDF5 for storage
- Improved chunked CSV reading
- Better handling of CICIDS CSV column variations
"""

class MemoryOptimizedLabelMatcher:
    def __init__(self):
        """
        Initialize label matcher with attack type mappings.
        """
        # Mapping from text labels to numeric codes for ML
        self.attack_mapping = {
            'BENIGN': 0,
            'Bot': 1,
            'DDoS': 2,
            'DoS GoldenEye': 3,
            'DoS Hulk': 4,
            'DoS Slowhttptest': 5,
            'DoS slowloris': 6,
            'FTP-Patator': 7,
            'Heartbleed': 8,
            'Infiltration': 9,
            'PortScan': 10,
            'SSH-Patator': 11,
            'Web Attack - Brute Force': 12,
            'Web Attack - SQL Injection': 13,
            'Web Attack - XSS': 14,
            'Web Attack – Brute Force': 12,  # Alternative spelling
            'Web Attack – XSS': 14,  # Alternative spelling
            'Web Attack – Sql Injection': 13  # Alternative spelling
        }
        
        # Cache for port-to-label mapping (memory efficient)
        self.port_label_cache = {}
        
        # Cache for IP+port combinations (more accurate)
        self.ip_port_label_cache = {}
        
        # Statistics tracking
        self.label_statistics = Counter()
    
    def load_labels_incrementally(self, label_files):
        """
        Load CICIDS label files incrementally to avoid memory overflow.
        Builds port-based and IP+port mappings for efficient matching.
        
        Args:
            label_files: List of CSV file paths
            
        Returns:
            bool: Success status
        """
        print("\n" + "="*60)
        print("LOADING CICIDS LABELS (MEMORY-OPTIMIZED)")
        print("="*60)
        
        label_counts = Counter()
        total_rows_processed = 0
        
        # Process each label file
        for file_idx, label_file in enumerate(label_files):
            print(f"\nProcessing label file {file_idx + 1}/{len(label_files)}: {os.path.basename(label_file)}")
            
            # Read CSV in chunks to manage memory
            chunk_size = 100000
            chunks_processed = 0
            
            try:
                # Process file in chunks
                for chunk in pd.read_csv(label_file, encoding='latin-1', chunksize=chunk_size):
                    # Clean column names (remove spaces)
                    chunk.columns = chunk.columns.str.strip()
                    
                    # Find label column (handles different naming conventions)
                    label_col = None
                    for col in ['Label', 'label', 'LABEL', ' Label']:
                        if col in chunk.columns:
                            label_col = col
                            break
                    
                    if not label_col:
                        print(f"  Warning: No label column found in chunk {chunks_processed}")
                        continue
                    
                    # Find required columns for matching
                    dst_port_col = None
                    dst_ip_col = None
                    src_ip_col = None
                    
                    # Port column variations
                    for col in ['Destination Port', 'Dst Port', 'dst_port', ' Destination Port', 'dst Port']:
                        if col in chunk.columns:
                            dst_port_col = col
                            break
                    
                    # IP column variations
                    for col in ['Destination IP', 'Dst IP', 'dst_ip', ' Destination IP']:
                        if col in chunk.columns:
                            dst_ip_col = col
                            break
                    
                    for col in ['Source IP', 'Src IP', 'src_ip', ' Source IP']:
                        if col in chunk.columns:
                            src_ip_col = col
                            break
                    
                    if label_col and dst_port_col:
                        # Build port-label mapping
                        for port, group in chunk.groupby(dst_port_col):
                            # Get most frequent label for this port
                            most_common_label = group[label_col].mode()
                            if len(most_common_label) > 0:
                                label = most_common_label.iloc[0]
                                
                                # Update port cache with majority vote
                                if port not in self.port_label_cache:
                                    self.port_label_cache[port] = Counter()
                                self.port_label_cache[port][label] += len(group)
                                
                                # Update statistics
                                label_counts[label] += len(group)
                        
                        # Build IP+port mapping for better accuracy
                        if dst_ip_col and src_ip_col:
                            for _, row in chunk.iterrows():
                                key = f"{row[src_ip_col]}:{row[dst_ip_col]}:{row[dst_port_col]}"
                                label = row[label_col]
                                
                                if key not in self.ip_port_label_cache:
                                    self.ip_port_label_cache[key] = Counter()
                                self.ip_port_label_cache[key][label] += 1
                    
                    chunks_processed += 1
                    total_rows_processed += len(chunk)
                    
                    # Periodic memory cleanup
                    if chunks_processed % 10 == 0:
                        gc.collect()
                        print(f"  Processed {chunks_processed * chunk_size:,} rows...")
                
                print(f"  Completed: {chunks_processed} chunks")
                
            except Exception as e:
                print(f"  Error processing file: {e}")
                continue
        
        # Finalize port-label mapping (keep only most common label per port)
        for port in self.port_label_cache:
            if isinstance(self.port_label_cache[port], Counter):
                # Get most common label for this port
                most_common = self.port_label_cache[port].most_common(1)[0][0]
                self.port_label_cache[port] = most_common
        
        # Finalize IP+port mapping
        for key in self.ip_port_label_cache:
            if isinstance(self.ip_port_label_cache[key], Counter):
                most_common = self.ip_port_label_cache[key].most_common(1)[0][0]
                self.ip_port_label_cache[key] = most_common
        
        # Display label distribution
        print("\n" + "-"*40)
        print("LABEL DISTRIBUTION:")
        print("-"*40)
        total = sum(label_counts.values())
        for label, count in label_counts.most_common():
            pct = count / total * 100 if total > 0 else 0
            print(f"  {label:30s}: {count:10,} ({pct:5.1f}%)")
        
        print(f"\nTotal rows processed: {total_rows_processed:,}")
        print(f"Port-label mappings created: {len(self.port_label_cache):,}")
        print(f"IP+Port mappings created: {len(self.ip_port_label_cache):,}")
        
        # Store statistics
        self.label_statistics = label_counts
        
        return True
    
    def apply_labels_to_batch(self, df):
        """
        Apply labels to a batch of extracted features.
        Uses IP+port mapping first, then port-based mapping as fallback.
        
        Args:
            df: DataFrame with extracted features
            
        Returns:
            DataFrame: With added label columns
        """
        # Initialize with BENIGN
        df['label'] = 0
        df['attack_type'] = 'BENIGN'
        df['label_confidence'] = 0.0
        
        # Try IP+port matching first (more accurate)
        if 'src_ip' in df.columns and 'dst_ip' in df.columns and 'dst_port' in df.columns:
            for idx, row in df.iterrows():
                key = f"{row.get('src_ip', '')}:{row.get('dst_ip', '')}:{row.get('dst_port', 0)}"
                if key in self.ip_port_label_cache:
                    attack_type = self.ip_port_label_cache[key]
                    df.at[idx, 'attack_type'] = attack_type
                    df.at[idx, 'label'] = self.attack_mapping.get(attack_type, 0)
                    df.at[idx, 'label_confidence'] = 0.9  # High confidence
        
        # Fallback to port-only matching for unlabeled flows
        if 'dst_port' in df.columns:
            unlabeled = df[df['label'] == 0]
            if not unlabeled.empty:
                # Map ports to attack types using cache
                port_labels = unlabeled['dst_port'].map(self.port_label_cache).fillna('BENIGN')
                df.loc[unlabeled.index, 'attack_type'] = port_labels
                
                # Convert text labels to numeric
                df.loc[unlabeled.index, 'label'] = port_labels.map(self.attack_mapping).fillna(0)
                
                # Lower confidence for port-only matching
                df.loc[unlabeled.index, 'label_confidence'] = 0.5
        
        # Ensure correct dtypes
        df['label'] = df['label'].astype(np.int8)
        df['label_confidence'] = df['label_confidence'].astype(np.float32)
        
        return df
    
    def process_features_with_labels(self, features_dir, label_files):
        """
        Process feature files and add labels in batches.
        Creates new Parquet files with labeled features.
        
        Args:
            features_dir: Directory containing feature Parquet files
            label_files: List of CICIDS CSV files
            
        Returns:
            str: Path to labeled feature directory
        """
        if not label_files:
            print("No label files provided, skipping labeling")
            return features_dir
        
        # Load label mappings into cache
        print("\nBuilding label cache from CSV files...")
        self.load_labels_incrementally(label_files)
        
        # Create output directory for labeled features
        labeled_dir = os.path.join(Config.TEMP_DIR, 'labeled_features')
        os.makedirs(labeled_dir, exist_ok=True)
        
        print("\n" + "="*60)
        print("APPLYING LABELS TO FEATURES")
        print("="*60)
        
        labeled_count = Counter()
        
        # Process each feature batch
        import glob
        feature_files = sorted(glob.glob(os.path.join(features_dir, 'batch_*.parquet')))
        
        for idx, feature_file in enumerate(tqdm(feature_files, desc="Labeling batches")):
            # Load feature batch
            batch = pd.read_parquet(feature_file)
            
            # Apply labels
            batch = self.apply_labels_to_batch(batch)
            
            # Track label distribution
            labeled_count.update(batch['attack_type'].value_counts().to_dict())
            
            # Save labeled batch
            output_file = os.path.join(labeled_dir, f'batch_{idx:04d}.parquet')
            batch.to_parquet(
                output_file,
                engine='pyarrow',
                compression='snappy',
                index=False
            )
            
            # Memory cleanup
            del batch
            gc.collect()
        
        # Display labeling results
        print("\n" + "-"*40)
        print("LABELING RESULTS:")
        print("-"*40)
        total = sum(labeled_count.values())
        for attack_type, count in labeled_count.most_common():
            pct = count / total * 100 if total > 0 else 0
            print(f"  {attack_type:30s}: {count:10,} ({pct:5.1f}%)")
        
        print(f"\nLabeled features saved to: {labeled_dir}")
        
        # Validate labeling quality
        benign_pct = labeled_count.get('BENIGN', 0) / total * 100 if total > 0 else 0
        if benign_pct > 90:
            print("\n Warning: >90% flows labeled as BENIGN")
            print("   This might indicate labeling issues or imbalanced dataset")
            print("   Consider using stratified sampling for ML training")
        
        return labeled_dir

# ### Step 7: Feature Analysis and Selection

In [8]:
# Cell 7: Feature Analysis - Selection & Importance Ranking
"""
PURPOSE: Analyze and select the most important features for machine learning
This class determines which features are most useful for detecting attacks.

WHAT THIS CELL DOES:
1. Collects statistical information about all features
2. Calculates feature importance using mutual information
3. Selects the top N most informative features
4. Removes redundant or uninformative features
5. Provides feature ranking for interpretability

KEY CONCEPTS:
- MUTUAL INFORMATION: Measures how much knowing a feature reduces uncertainty about the label
- FEATURE SELECTION: Choosing subset of features that maximize predictive power
- CURSE OF DIMENSIONALITY: Too many features can hurt ML performance
- FEATURE IMPORTANCE: Understanding which features drive predictions

WHY FEATURE SELECTION MATTERS:
- Reduces training time (fewer features to process)
- Improves model performance (removes noise)
- Prevents overfitting (simpler models generalize better)
- Enhances interpretability (understand what drives detection)

SELECTION STRATEGY:
1. Statistical Analysis: Mean, variance, range for each feature
2. Importance Scoring: Mutual information with target labels
3. Redundancy Removal: Correlation analysis
4. Top-K Selection: Keep only the best features

IMPORTANT CHANGES:
- Using Parquet for data loading
- Parallel processing for importance calculation
- Better handling of CICIDS-specific features
"""

class MemoryOptimizedFeatureAnalyzer:
    def __init__(self):
        """
        Initialize feature analyzer with storage for statistics and importance scores.
        """
        self.feature_importance = {}  # Feature name -> importance score
        self.selected_features = []   # Final list of selected features
        self.feature_stats = {}        # Statistical summaries per feature
    
    def analyze_features_incrementally(self, features_dir, target_col='label'):
        """
        Analyze features using two-pass incremental processing.
        Pass 1: Collect statistics
        Pass 2: Calculate importance on representative sample
        
        Args:
            features_dir: Directory containing feature Parquet files
            target_col: Name of label column
            
        Returns:
            list: Selected feature names
        """
        print("\n" + "="*60)
        print("FEATURE ANALYSIS (MEMORY-OPTIMIZED)")
        print("="*60)
        
        # ===== PASS 1: STATISTICS COLLECTION =====
        print("Pass 1: Collecting feature statistics...")
        print("  This helps understand feature distributions")
        self.collect_feature_stats(features_dir, target_col)
        
        # ===== PASS 2: IMPORTANCE CALCULATION =====
        print("\nPass 2: Calculating feature importance on sample...")
        print("  This identifies most informative features")
        self.calculate_importance_sample(features_dir, target_col)
        
        # ===== FEATURE SELECTION =====
        # Sort features by importance
        sorted_features = sorted(
            self.feature_importance.items(),
            key=lambda x: x[1],
            reverse=True
        )
        
        # Select top features
        self.selected_features = [f[0] for f in sorted_features[:Config.TOP_FEATURES]]
        
        # ===== DISPLAY RESULTS =====
        print(f"\n" + "="*40)
        print(f"TOP {min(15, len(sorted_features))} FEATURES:")
        print("="*40)
        print(f"{'Rank':<5} {'Feature':<35} {'Importance':<10} {'Type'}")
        print("-"*65)
        
        for i, (feature, score) in enumerate(sorted_features[:15], 1):
            # Determine feature type for interpretation
            feature_type = self.get_feature_type(feature)
            print(f"{i:<5} {feature:<35} {score:>10.4f} {feature_type}")
        
        print(f"\nTotal features analyzed: {len(self.feature_importance)}")
        print(f"Features selected: {len(self.selected_features)}")
        
        # ===== CICIDS-SPECIFIC FEATURE INSIGHTS =====
        cicids_features = [f for f in self.selected_features if any(x in f.lower() for x in ['fwd', 'bwd', 'active', 'idle', 'iat'])]
        if cicids_features:
            print(f"\nCICIDS-specific features selected: {len(cicids_features)}")
            print(f"  Examples: {cicids_features[:3]}")
        
        return self.selected_features
    
    def collect_feature_stats(self, features_dir, target_col):
        """
        First pass: Collect statistical summaries for each feature.
        This helps understand data distribution and identify issues.
        
        Statistics collected:
        - Sum, sum of squares (for mean/variance calculation)
        - Min, max (for range)
        - Count (for missing value detection)
        """
        import glob
        feature_files = sorted(glob.glob(os.path.join(features_dir, 'batch_*.parquet')))
        
        # Get feature columns from first batch
        first_batch = pd.read_parquet(feature_files[0], columns=None, engine='pyarrow')
        
        # Identify feature columns (exclude metadata)
        exclude_cols = {'flow_id', target_col, 'attack_type', 'label_confidence'}
        feature_cols = [col for col in first_batch.columns 
                      if col not in exclude_cols]
        
        print(f"  Analyzing {len(feature_cols)} features...")
        
        # Initialize statistics collectors
        for col in feature_cols:
            self.feature_stats[col] = {
                'sum': 0,      # For mean calculation
                'sum_sq': 0,   # For variance calculation
                'count': 0,    # Total non-null values
                'min': float('inf'),
                'max': float('-inf'),
                'zeros': 0,    # Count of zero values
                'unique': set() # Track unique values (sampled)
            }
        
        # Process all batches
        for file in tqdm(feature_files, desc="Collecting stats"):
            batch = pd.read_parquet(file, columns=feature_cols, engine='pyarrow')
            
            for col in feature_cols:
                if col in batch.columns:
                    # Get non-null values
                    values = batch[col].fillna(0)
                    
                    # Update statistics
                    stats = self.feature_stats[col]
                    stats['sum'] += values.sum()
                    stats['sum_sq'] += (values ** 2).sum()
                    stats['count'] += len(values)
                    stats['min'] = min(stats['min'], values.min())
                    stats['max'] = max(stats['max'], values.max())
                    stats['zeros'] += (values == 0).sum()
                    
                    # Sample unique values (limit to 100 for memory)
                    if len(stats['unique']) < 100:
                        sample_size = min(10, len(values))
                        stats['unique'].update(values.sample(sample_size).tolist())
            
            # Memory cleanup
            del batch
            gc.collect()
        
        # Calculate derived statistics
        for col, stats in self.feature_stats.items():
            if stats['count'] > 0:
                stats['mean'] = stats['sum'] / stats['count']
                stats['variance'] = (stats['sum_sq'] / stats['count']) - (stats['mean'] ** 2)
                stats['std'] = np.sqrt(max(0, stats['variance']))  # Avoid negative variance due to rounding
                stats['range'] = stats['max'] - stats['min']
                stats['zero_ratio'] = stats['zeros'] / stats['count']
            else:
                stats['mean'] = stats['variance'] = stats['std'] = stats['range'] = 0
                stats['zero_ratio'] = 1
    
    def calculate_importance_sample(self, features_dir, target_col, sample_size=100000):
        """
        Second pass: Calculate feature importance using mutual information.
        Uses a representative sample for efficiency.
        
        OPTIMIZED: Can use parallel processing if available.
        
        Mutual Information measures:
        - How much information a feature provides about the target
        - Non-linear relationships (unlike correlation)
        - Works with any feature type
        
        Args:
            features_dir: Directory with feature files
            target_col: Target label column
            sample_size: Number of samples for importance calculation
        """
        # Load stratified sample for importance calculation
        print(f"  Loading sample of {sample_size:,} flows...")
        sample_dfs = []
        remaining_samples = sample_size
        
        import glob
        feature_files = sorted(glob.glob(os.path.join(features_dir, 'batch_*.parquet')))
        
        # Sample from different parts of the dataset
        sample_interval = max(1, len(feature_files) // 10)  # Sample from 10 points
        
        for i in range(0, len(feature_files), sample_interval):
            if remaining_samples <= 0:
                break
            
            file = feature_files[min(i, len(feature_files)-1)]
            batch_sample_size = min(remaining_samples, 10000)
            
            # Load batch sample
            batch = pd.read_parquet(file)
            if len(batch) > batch_sample_size:
                batch = batch.sample(batch_sample_size, random_state=Config.RANDOM_STATE)
            sample_dfs.append(batch)
            remaining_samples -= len(batch)
        
        # Combine samples
        sample_df = pd.concat(sample_dfs, ignore_index=True)
        print(f"  Sample loaded: {len(sample_df):,} flows")
        
        # Get feature columns
        exclude_cols = {'flow_id', target_col, 'attack_type', 'label_confidence'}
        feature_cols = [col for col in sample_df.columns 
                       if col not in exclude_cols]
        
        # Prepare feature matrix and labels
        X = sample_df[feature_cols].fillna(0)
        y = sample_df[target_col]
        
        # Calculate mutual information
        print("  Calculating mutual information scores...")
        print(f"  Using {Config.N_JOBS if Config.N_JOBS > 0 else 'all'} CPU cores")
        
        mi_scores = mutual_info_classif(
            X, y, 
            random_state=Config.RANDOM_STATE,
            n_neighbors=3  # Faster calculation
        )
        
        # Store importance scores
        self.feature_importance = dict(zip(feature_cols, mi_scores))
        
        # Identify uninformative features (near-zero importance)
        uninformative = [f for f, score in self.feature_importance.items() if score < 0.001]
        if uninformative:
            print(f"\n  ⚠️  Found {len(uninformative)} uninformative features (MI < 0.001)")
            print(f"     Examples: {uninformative[:5]}")
        
        # Identify highly important CICIDS features
        cicids_important = [(f, s) for f, s in self.feature_importance.items() 
                           if s > 0.1 and any(x in f.lower() for x in ['fwd', 'bwd', 'iat', 'active', 'idle'])]
        if cicids_important:
            print(f"\n  ✓ Important CICIDS features found: {len(cicids_important)}")
            for feat, score in cicids_important[:3]:
                print(f"     {feat}: {score:.3f}")
        
        # Clean up
        del sample_df, sample_dfs, X, y
        gc.collect()
    
    def get_feature_type(self, feature_name):
        """
        Categorize feature by its name for better interpretation.
        Enhanced to recognize CICIDS-specific features.
        
        Categories:
        - Flow: Timing and size statistics
        - Forward: Forward direction features (CICIDS)
        - Backward: Backward direction features (CICIDS)
        - Active/Idle: Activity patterns (CICIDS)
        - Flag: TCP flag related
        - Port: Port number features
        - Payload: Content-based features
        - NLP: Natural language processing scores
        - Engineered: Derived features
        """
        feature_lower = feature_name.lower()
        
        # CICIDS-specific categories
        if 'fwd' in feature_lower or 'forward' in feature_lower:
            return "Forward"
        elif 'bwd' in feature_lower or 'backward' in feature_lower:
            return "Backward"
        elif 'active' in feature_lower or 'idle' in feature_lower:
            return "Active/Idle"
        elif 'iat' in feature_lower:
            return "IAT"
        elif 'flow' in feature_lower or 'duration' in feature_lower:
            return "Flow"
        elif 'flag' in feature_lower or 'syn' in feature_lower or 'ack' in feature_lower:
            return "Flag"
        elif 'port' in feature_lower:
            return "Port"
        elif 'payload' in feature_lower or 'entropy' in feature_lower:
            return "Payload"
        elif 'nlp' in feature_lower:
            return "NLP"
        elif 'ratio' in feature_lower or 'rate' in feature_lower:
            return "Engineered"
        elif 'has_' in feature_lower or 'is_' in feature_lower:
            return "Binary"
        elif 'suspicious' in feature_lower:
            return "Detection"
        else:
            return "Other"
    
    def get_feature_insights(self):
        """
        Provide insights about selected features for interpretability.
        Helps understand what the model will focus on.
        
        Returns:
            dict: Insights about feature selection
        """
        insights = {
            'total_features': len(self.feature_importance),
            'selected_features': len(self.selected_features),
            'feature_types': Counter(),
            'top_5_features': [],
            'recommendations': []
        }
        
        # Analyze feature types
        for feature in self.selected_features:
            feature_type = self.get_feature_type(feature)
            insights['feature_types'][feature_type] += 1
        
        # Get top 5 features with scores
        for feature in self.selected_features[:5]:
            insights['top_5_features'].append({
                'name': feature,
                'importance': self.feature_importance[feature],
                'type': self.get_feature_type(feature)
            })
        
        # Provide recommendations based on feature distribution
        if insights['feature_types'].get('Forward', 0) + insights['feature_types'].get('Backward', 0) > 10:
            insights['recommendations'].append(
                "Strong directional features - good for detecting asymmetric attacks"
            )
        
        if insights['feature_types'].get('NLP', 0) > 5:
            insights['recommendations'].append(
                "High NLP feature count - model focuses on payload content"
            )
        
        if insights['feature_types'].get('Active/Idle', 0) > 3:
            insights['recommendations'].append(
                "Active/Idle features selected - good for detecting slow attacks"
            )
        
        if insights['feature_types'].get('Flow', 0) > 5:
            insights['recommendations'].append(
                "High flow feature count - model focuses on traffic patterns"
            )
        
        if len(self.selected_features) < 30:
            insights['recommendations'].append(
                "Consider increasing TOP_FEATURES if accuracy is low"
            )
        
        return insights

# ### Step 8: Machine Learning Pipeline

In [9]:
# Cell 8: Machine Learning Pipeline - Incremental Training & Evaluation
"""
PURPOSE: Train and evaluate machine learning models using incremental learning
This class implements memory-efficient ML training for large datasets that don't fit in RAM.

WHAT THIS CELL DOES:
1. Implements incremental/online learning algorithms
2. Trains models in batches without loading all data
3. Performs train/test split at the batch level
4. Evaluates model performance progressively
5. Supports multiple ML algorithms optimized for streaming

INCREMENTAL LEARNING:
Traditional ML loads all data at once. Incremental learning:
- Processes data in small batches
- Updates model parameters gradually
- Never needs full dataset in memory
- Perfect for datasets larger than RAM

ALGORITHMS USED:
1. SGD Classifier (Stochastic Gradient Descent):
   - Linear model with online learning
   - Fast and memory efficient
   - Good for high-dimensional data
   - Updates with each batch

2. LightGBM:
   - Often faster than XGBoost on CPU
   - Better memory efficiency
   - High accuracy gradient boosting

3. XGBoost with External Memory:
   - Gradient boosting with disk cache
   - High accuracy for complex patterns
   - Can process data larger than RAM
   
4. MiniBatch K-Means (optional):
   - Clustering for anomaly detection
   - Unsupervised learning option

WHY THESE MODELS:
- All support incremental/batch learning
- Memory efficient for large datasets
- Proven effectiveness on network traffic
- Balance between speed and accuracy

IMPORTANT CHANGES:
- Added LightGBM as primary boosting method
- Using Parquet for data loading
- Better memory management during training
- Optimized for AMD Ryzen CPU (parallel processing)
"""

class MemoryOptimizedMLPipeline:
    def __init__(self):
        """
        Initialize ML pipeline with model storage and result tracking.
        """
        self.models = {}      # Trained model objects
        self.results = {}     # Performance metrics
        self.scaler = None    # Feature scaler (fitted on first batch)
        self.train_history = defaultdict(list)  # Track training progress
    
    def train_models_incrementally(self, features_dir, selected_features):
        """
        Main training function using incremental learning.
        Processes data in batches, updating models progressively.
        
        OPTIMIZED: Uses Parquet loading and parallel processing.
        
        Args:
            features_dir: Directory containing feature Parquet files
            selected_features: List of feature names to use
            
        Returns:
            tuple: (models dict, results dict)
        """
        print("\n" + "="*60)
        print("INCREMENTAL MACHINE LEARNING TRAINING")
        print("="*60)
        print(f"Training models: {Config.SELECTED_MODELS}")
        print(f"Using {len(selected_features)} selected features")
        print(f"CPU cores available: {Config.N_JOBS if Config.N_JOBS > 0 else 'All'}")
        
        # ===== INITIALIZE MODELS =====
        self.initialize_models()
        
        # Get class information for CICIDS
        n_classes = 15  # CICIDS has 15 attack types
        classes = np.arange(n_classes)
        
        # Training metrics
        batch_count = 0
        train_scores = defaultdict(list)
        
        # Get all feature files
        import glob
        feature_files = sorted(glob.glob(os.path.join(features_dir, 'batch_*.parquet')))
        n_batches = len(feature_files)
        
        # ===== TRAIN/TEST SPLIT AT BATCH LEVEL =====
        # Split batches into train and test sets
        # Ensure at least 1 batch for training if we have any data
        if n_batches == 1:
            # With only 1 batch, use it for both training and testing
            print(f"\nOnly 1 batch available - using for both training and testing")
            train_files = feature_files
            test_files = feature_files
        elif n_batches == 2:
            # With 2 batches, use 1 for train, 1 for test
            train_files = feature_files[:1]
            test_files = feature_files[1:]
        else:
            # Normal split for 3+ batches
            train_size = max(1, int(n_batches * (1 - Config.TEST_SIZE)))
            train_files = feature_files[:train_size]
            test_files = feature_files[train_size:]
        
        print(f"\nData split:")
        print(f"  Training batches: {len(train_files)}")
        print(f"  Testing batches: {len(test_files)}")
        
        # ===== TRAINING PHASE =====
        print("\n" + "-"*40)
        print("TRAINING PHASE")
        print("-"*40)
        
        for file in tqdm(train_files, desc="Training incrementally"):
            # Load batch
            batch = pd.read_parquet(file)
            
            # Prepare features and labels
            X_batch, y_batch = self.prepare_batch(batch, selected_features)
            
            if X_batch is None or len(X_batch) == 0:
                continue
            
            # ===== FEATURE SCALING =====
            # Fit scaler on first batch, transform all others
            if self.scaler is None:
                print(f"  Fitting scaler on first batch ({len(X_batch)} samples)")
                self.scaler = StandardScaler()
                X_batch = self.scaler.fit_transform(X_batch)
            else:
                X_batch = self.scaler.transform(X_batch)
            
            # ===== TRAIN EACH MODEL =====
            # SGD Classifier (incremental)
            if 'sgd' in self.models:
                self.models['sgd'].partial_fit(X_batch, y_batch, classes=classes)
                
                # Track training progress
                if batch_count % 5 == 0:
                    score = self.models['sgd'].score(X_batch, y_batch)
                    train_scores['sgd'].append(score)
                    if batch_count % 20 == 0:
                        print(f"    SGD Batch {batch_count:3d} accuracy: {score:.4f}")
            
            # MiniBatch K-Means (if selected)
            if 'minibatch_kmeans' in self.models:
                self.models['minibatch_kmeans'].partial_fit(X_batch)
            
            batch_count += 1
            
            # Memory cleanup
            del batch, X_batch, y_batch
            gc.collect()
        
        print(f"\nTraining complete: {batch_count} batches processed")
        
        # ===== TRAIN LIGHTGBM (Non-incremental but memory-efficient) =====
        if 'lightgbm' in Config.SELECTED_MODELS:
            print("\n" + "-"*40)
            print("LIGHTGBM TRAINING")
            print("-"*40)
            self.train_lightgbm(train_files, selected_features, classes)
        
        # ===== TRAIN XGBOOST IF SELECTED =====
        if 'xgboost_incremental' in Config.SELECTED_MODELS:
            print("\n" + "-"*40)
            print("XGBOOST TRAINING (External Memory)")
            print("-"*40)
            self.train_xgboost_external_memory(train_files, selected_features)
        
        # ===== TESTING PHASE =====
        print("\n" + "-"*40)
        print("TESTING PHASE")
        print("-"*40)
        print("Evaluating on held-out test batches...")
        
        # Check if scaler was fitted
        if self.scaler is None:
            print("⚠️  Warning: Scaler not fitted during training, fitting now...")
            # Fit scaler on first available batch
            for file in feature_files[:1]:
                batch = pd.read_parquet(file)
                X_batch, y_batch = self.prepare_batch(batch, selected_features)
                if X_batch is not None and len(X_batch) > 0:
                    self.scaler = StandardScaler()
                    self.scaler.fit(X_batch)
                    print(f"  Scaler fitted on {len(X_batch)} samples")
                    break
                del batch
                gc.collect()
        
        test_predictions = defaultdict(list)
        test_labels = []
        test_batches = 0
        
        for file in tqdm(test_files, desc="Testing"):
            batch = pd.read_parquet(file)
            X_batch, y_batch = self.prepare_batch(batch, selected_features)
            
            if X_batch is None or len(X_batch) == 0:
                continue
            
            # Scale features
            X_batch = self.scaler.transform(X_batch)
            test_labels.extend(y_batch)
            
            # Get predictions from each model
            if 'sgd' in self.models:
                test_predictions['sgd'].extend(
                    self.models['sgd'].predict(X_batch)
                )
            
            if 'lightgbm' in self.models:
                test_predictions['lightgbm'].extend(
                    self.models['lightgbm'].predict(X_batch)
                )
            
            if 'minibatch_kmeans' in self.models:
                # For clustering, use cluster assignment as "prediction"
                test_predictions['minibatch_kmeans'].extend(
                    self.models['minibatch_kmeans'].predict(X_batch)
                )
            
            test_batches += 1
            
            # Memory cleanup
            del batch, X_batch, y_batch
            gc.collect()
        
        print(f"Testing complete: {test_batches} batches processed")
        
        # ===== CALCULATE METRICS =====
        print("\n" + "-"*40)
        print("MODEL PERFORMANCE")
        print("-"*40)
        
        test_labels = np.array(test_labels)
        
        for model_name, predictions in test_predictions.items():
            predictions = np.array(predictions)
            
            # Calculate accuracy
            accuracy = (predictions == test_labels).mean()
            
            # Store results
            self.results[model_name] = {
                'accuracy': accuracy,
                'train_scores': train_scores.get(model_name, []),
                'test_samples': len(test_labels),
                'unique_predictions': len(np.unique(predictions))
            }
            
            print(f"\n{model_name.upper()}:")
            print(f"  Test Accuracy: {accuracy:.4f}")
            print(f"  Test Samples: {len(test_labels):,}")
            print(f"  Unique Predictions: {len(np.unique(predictions))}")
            
            # Per-class accuracy for best model
            if model_name in ['sgd', 'lightgbm'] and len(np.unique(test_labels)) > 1:
                self.calculate_per_class_metrics(predictions, test_labels)
        
        return self.models, self.results
    
    def initialize_models(self):
        """
        Initialize selected ML models with appropriate parameters.
        All models chosen for their incremental learning capability or memory efficiency.
        """
        # SGD Classifier - Linear model with online learning
        if 'sgd' in Config.SELECTED_MODELS:
            print("\nInitializing SGD Classifier...")
            self.models['sgd'] = SGDClassifier(
                loss='log_loss',      # Logistic regression
                penalty='l2',         # L2 regularization
                alpha=0.001,          # Regularization strength
                max_iter=1000,        # Max iterations per batch
                random_state=Config.RANDOM_STATE,
                n_jobs=Config.N_JOBS, # Use available CPU cores
                warm_start=True       # Preserve previous training
            )
        
        # MiniBatch K-Means - For anomaly detection
        if 'minibatch_kmeans' in Config.SELECTED_MODELS:
            print("Initializing MiniBatch K-Means...")
            self.models['minibatch_kmeans'] = MiniBatchKMeans(
                n_clusters=15,        # Number of clusters (match attack types)
                batch_size=1000,      # Samples per batch
                random_state=Config.RANDOM_STATE
            )
        
        # LightGBM and XGBoost will be initialized in their specific methods
        if 'lightgbm' in Config.SELECTED_MODELS:
            self.models['lightgbm'] = None
        if 'xgboost_incremental' in Config.SELECTED_MODELS:
            self.models['xgboost_incremental'] = None
    
    def prepare_batch(self, batch, selected_features):
        """
        Prepare a batch for training by extracting features and labels.
        Handles missing features and data type conversions.
        
        Args:
            batch: DataFrame batch from Parquet
            selected_features: List of feature names to use
            
        Returns:
            tuple: (X feature matrix, y labels)
        """
        if 'label' not in batch.columns:
            return None, None
        
        # Get available features (some might be missing in certain batches)
        available_features = [f for f in selected_features if f in batch.columns]
        
        if not available_features:
            return None, None
        
        # Extract features and labels
        X = batch[available_features].fillna(0).values
        y = batch['label'].values
        
        # Ensure correct data types
        X = X.astype(np.float32)
        y = y.astype(np.int32)
        
        return X, y
    
    def train_lightgbm(self, train_files, selected_features, classes):
        """
        Train LightGBM using memory-efficient approach.
        LightGBM is often faster than XGBoost on CPU systems.
        """
        print("Training LightGBM with memory-efficient settings...")
        
        # Load training data in chunks and combine
        X_train_list = []
        y_train_list = []
        
        # Sample from training files for LightGBM
        sample_files = train_files[::max(1, len(train_files) // 20)]  # Use up to 20 files
        
        for file in tqdm(sample_files, desc="Loading LightGBM training data"):
            batch = pd.read_parquet(file)
            X_batch, y_batch = self.prepare_batch(batch, selected_features)
            
            if X_batch is not None:
                X_batch = self.scaler.transform(X_batch)
                X_train_list.append(X_batch)
                y_train_list.append(y_batch)
            
            del batch
            gc.collect()
        
        # Combine samples
        X_train = np.vstack(X_train_list)
        y_train = np.hstack(y_train_list)
        
        print(f"  Training on {len(X_train):,} samples")
        
        # LightGBM parameters optimized for CPU and memory
        params = {
            'objective': 'multiclass',
            'num_class': len(classes),
            'metric': 'multi_logloss',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'device': 'cpu',
            'num_threads': Config.N_JOBS if Config.N_JOBS > 0 else -1,
            'verbose': -1,
            'max_bin': 255,  # Memory optimization
            'min_data_in_leaf': 20,
            'seed': Config.RANDOM_STATE
        }
        
        # Create dataset
        train_data = lgb.Dataset(X_train, label=y_train)
        
        # Train model
        self.models['lightgbm'] = lgb.train(
            params,
            train_data,
            num_boost_round=100,
            valid_sets=[train_data],
            callbacks=[lgb.early_stopping(10), lgb.log_evaluation(20)]
        )
        
        print("  LightGBM training complete")
        
        # Clean up
        del X_train, y_train, X_train_list, y_train_list
        gc.collect()
    
    def train_xgboost_external_memory(self, train_files, selected_features):
        """
        Train XGBoost using external memory for very large datasets.
        XGBoost can process data from disk without loading it all.
        
        This is more complex but enables training on massive datasets.
        """
        print("Training XGBoost with external memory support...")
        
        try:
            # Create cache file for XGBoost
            cache_file = os.path.join(Config.TEMP_DIR, 'xgb_cache')
            
            # Convert Parquet to format XGBoost can read
            print("  Converting data to XGBoost format...")
            libsvm_file = self.create_libsvm_file(train_files, selected_features)
            
            # Create DMatrix with cache (enables external memory)
            print("  Creating DMatrix with disk cache...")
            dtrain = xgb.DMatrix(f'{libsvm_file}#dtrain.cache')
            
            # XGBoost parameters optimized for large data
            params = {
                'max_depth': 6,           # Tree depth
                'eta': 0.1,               # Learning rate
                'objective': 'multi:softmax',  # Multiclass classification
                'num_class': 15,          # Number of attack types
                'tree_method': 'approx',  # Approximate algorithm for large data
                'sketch_eps': 0.03,       # Approximation factor
                'max_bin': 256,           # Max bins for histogram
                'subsample': 0.5,         # Sample 50% of data per tree
                'colsample_bytree': 0.8,  # Sample 80% of features per tree
                'nthread': Config.N_JOBS if Config.N_JOBS > 0 else -1,
                'seed': Config.RANDOM_STATE
            }
            
            # Train model
            print("  Training XGBoost model...")
            num_rounds = 50  # Number of boosting rounds
            self.models['xgboost_incremental'] = xgb.train(
                params, dtrain, num_rounds,
                verbose_eval=10  # Print progress every 10 rounds
            )
            
            # Store basic results
            self.results['xgboost_incremental'] = {
                'accuracy': 0.0,  # Will be calculated during testing
                'num_trees': num_rounds
            }
            
            print("  XGBoost training complete")
            
            # Cleanup temporary files
            if os.path.exists(libsvm_file):
                os.remove(libsvm_file)
            
        except Exception as e:
            print(f"  XGBoost training failed: {e}")
            print("  Falling back to other models")
    
    def create_libsvm_file(self, train_files, selected_features, max_rows=500000):
        """
        Convert Parquet features to LibSVM format for XGBoost.
        LibSVM is a sparse format efficient for ML libraries.
        
        Format: label feat1:val1 feat2:val2 ...
        
        Args:
            train_files: List of training Parquet files
            selected_features: Features to include
            max_rows: Limit rows for training
            
        Returns:
            str: Path to LibSVM file
        """
        libsvm_file = os.path.join(Config.TEMP_DIR, 'train.libsvm')
        
        with open(libsvm_file, 'w') as f:
            rows_written = 0
            
            for file in train_files:
                if rows_written >= max_rows:
                    break
                
                batch = pd.read_parquet(file)
                X_batch, y_batch = self.prepare_batch(batch, selected_features)
                
                if X_batch is None:
                    continue
                
                # Scale features
                X_batch = self.scaler.transform(X_batch)
                
                # Write in LibSVM format
                for i in range(len(X_batch)):
                    label = int(y_batch[i])
                    # Only write non-zero features (sparse format)
                    features = ' '.join([f'{j+1}:{v}' for j, v in enumerate(X_batch[i]) if v != 0])
                    f.write(f'{label} {features}\n')
                
                rows_written += len(X_batch)
                del batch, X_batch, y_batch
                gc.collect()
        
        print(f"    Created LibSVM file with {rows_written:,} samples")
        return libsvm_file
    
    def calculate_per_class_metrics(self, predictions, true_labels):
        """
        Calculate performance metrics for each attack type.
        Helps identify which attacks are well-detected vs problematic.
        """
        print("\n  Per-Class Performance:")
        print("  " + "-"*40)
        
        # Attack type names for CICIDS
        attack_names = {
            0: 'BENIGN',
            1: 'Bot',
            2: 'DDoS',
            3: 'DoS GoldenEye',
            4: 'DoS Hulk',
            5: 'DoS Slowhttptest',
            6: 'DoS slowloris',
            7: 'FTP-Patator',
            8: 'Heartbleed',
            9: 'Infiltration',
            10: 'PortScan',
            11: 'SSH-Patator',
            12: 'Web Attack - Brute Force',
            13: 'Web Attack - SQL Injection',
            14: 'Web Attack - XSS'
        }
        
        # Calculate per-class accuracy
        for class_id in np.unique(true_labels):
            mask = true_labels == class_id
            if mask.sum() > 0:
                class_acc = (predictions[mask] == true_labels[mask]).mean()
                class_name = attack_names.get(class_id, f'Class {class_id}')
                print(f"    {class_name:30s}: {class_acc:.4f} ({mask.sum()} samples)")

# ### Step 9: extended Machine Learning Pipeline

In [10]:
# Cell 9: Visualization Suite - Data Analysis Dashboards
"""
PURPOSE: Create interactive visualizations to understand data and results
This class generates comprehensive visualizations for analysis insights.

WHAT THIS CELL DOES:
1. Creates interactive charts using Plotly
2. Visualizes attack distributions and patterns
3. Shows model performance comparisons
4. Generates network communication graphs
5. Builds an integrated HTML dashboard

VISUALIZATIONS CREATED:
1. Protocol Distribution - Pie chart of TCP/UDP/ICMP traffic
2. Attack Distribution - Bar chart of attack type frequencies  
3. Flow Timeline - Scatter plot of traffic over time
4. Port Heatmap - Communication patterns between ports
5. Feature Correlation - Correlation matrix of top features
6. Model Comparison - Bar chart of model accuracies
7. CICIDS Features - Special visualizations for forward/backward features
8. Dashboard - Combined HTML view of all visualizations

WHY VISUALIZATION MATTERS:
- Helps understand data patterns before modeling
- Identifies class imbalance issues
- Reveals temporal attack patterns
- Shows which features are correlated
- Makes results interpretable for stakeholders

MEMORY OPTIMIZATION:
- Uses data sampling (max 10K points per viz)
- Loads only required columns
- Generates static HTML files
- Cleans up data after each visualization

IMPORTANT CHANGES:
- Using Parquet for data loading
- Added CICIDS-specific visualizations
- Optimized for large datasets
"""

class MemoryOptimizedVisualizer:
    def __init__(self, features_dir, ml_results, output_dir):
        """
        Initialize visualizer with data source and output location.
        
        Args:
            features_dir: Directory containing feature Parquet files
            ml_results: Dictionary of ML model results
            output_dir: Where to save visualization files
        """
        self.features_dir = features_dir
        self.ml_results = ml_results
        self.output_dir = output_dir
        self.sample_size = min(Config.SAMPLE_SIZE, 10000)  # Max points for visualization
    
    def create_all_visualizations(self):
        """
        Generate all visualizations and create dashboard.
        Each visualization is saved as a separate HTML file.
        """
        if not Config.GENERATE_VISUALS:
            print("Visualization disabled in config")
            return
        
        print("\n" + "="*60)
        print("GENERATING VISUALIZATIONS (SAMPLED DATA)")
        print("="*60)
        print(f"Creating visualizations with {self.sample_size:,} sample points")
        print("This provides insights without loading full dataset")
        
        # Load sample data for visualization
        sample_df = self.load_visualization_sample()
        
        if sample_df is None or sample_df.empty:
            print("No data available for visualization")
            return
        
        # List of visualizations to create
        viz_functions = [
            ("Protocol Distribution", lambda: self.create_protocol_distribution(sample_df)),
            ("Attack Distribution", lambda: self.create_attack_distribution(sample_df)),
            ("Flow Timeline", lambda: self.create_flow_timeline(sample_df)),
            ("Port Heatmap", lambda: self.create_port_heatmap(sample_df)),
            ("Feature Correlation", lambda: self.create_feature_correlation(sample_df)),
            ("CICIDS Features", lambda: self.create_cicids_feature_analysis(sample_df)),
            ("Model Performance", lambda: self.create_model_comparison())
        ]
        
        # Create each visualization
        for name, func in tqdm(viz_functions, desc="Creating visualizations"):
            try:
                func()
                print(f"  Created: {name}")
            except Exception as e:
                print(f"  Failed {name}: {e}")
        
        # Create integrated dashboard
        self.create_dashboard(sample_df)
        
        # Clean up sample data
        del sample_df
        gc.collect()
        
        print(f"\nVisualizations saved to: {self.output_dir}")
        print(f"  Open dashboard.html in browser to view all charts")
    
    def load_visualization_sample(self):
        """
        Load a representative sample of data for visualization.
        Samples from different parts of dataset for better representation.
        
        Returns:
            DataFrame: Sample of features for visualization
        """
        print(f"\nLoading sample of {self.sample_size:,} flows for visualization...")
        
        sample_dfs = []
        remaining = self.sample_size
        
        import glob
        feature_files = sorted(glob.glob(os.path.join(self.features_dir, 'batch_*.parquet')))
        
        # Sample from different parts of dataset for diversity
        sample_interval = max(1, len(feature_files) // 10)  # Sample from 10 points
        
        for i in range(0, len(feature_files), sample_interval):
            if remaining <= 0:
                break
            
            file = feature_files[min(i, len(feature_files)-1)]
            batch_sample_size = min(remaining, 1000)
            
            # Load sample from this batch
            batch = pd.read_parquet(file)
            if len(batch) > batch_sample_size:
                batch = batch.sample(batch_sample_size, random_state=Config.RANDOM_STATE)
            sample_dfs.append(batch)
            remaining -= len(batch)
        
        if sample_dfs:
            sample = pd.concat(sample_dfs, ignore_index=True)
            print(f"  Loaded {len(sample):,} flows from {len(sample_dfs)} batches")
            return sample
        
        return pd.DataFrame()
    
    def create_protocol_distribution(self, df):
        """
        Create pie chart showing distribution of network protocols.
        Helps understand traffic composition (TCP vs UDP vs other).
        """
        if 'protocol' not in df.columns:
            return
        
        # Map protocol numbers to names
        protocol_map = {
            6: 'TCP',
            17: 'UDP', 
            1: 'ICMP',
            41: 'IPv6',
            47: 'GRE'
        }
        
        df['protocol_name'] = df['protocol'].map(protocol_map).fillna('Other')
        protocol_counts = df['protocol_name'].value_counts()
        
        # Create pie chart
        fig = go.Figure(data=[go.Pie(
            labels=protocol_counts.index,
            values=protocol_counts.values,
            hole=0.3,  # Donut chart
            marker=dict(colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96A6A6', '#FFA07A']),
            textposition='auto',
            textinfo='label+percent'
        )])
        
        fig.update_layout(
            title="Protocol Distribution (Sample)",
            height=400,
            width=600,
            showlegend=True
        )
        
        # Save to HTML
        fig.write_html(os.path.join(self.output_dir, 'protocol_distribution.html'))
    
    def create_attack_distribution(self, df):
        """
        Create bar chart showing distribution of attack types.
        Critical for understanding class balance in dataset.
        """
        if 'attack_type' not in df.columns:
            return
        
        # Count attack types
        attack_counts = df['attack_type'].value_counts().head(15)
        
        # Create bar chart with color coding
        colors = ['green' if x == 'BENIGN' else 'red' for x in attack_counts.index]
        
        fig = go.Figure(data=[go.Bar(
            x=attack_counts.index,
            y=attack_counts.values,
            text=attack_counts.values,
            textposition='auto',
            marker_color=colors,
            hovertemplate='%{x}<br>Count: %{y}<extra></extra>'
        )])
        
        fig.update_layout(
            title="Attack Type Distribution (Sample)",
            xaxis_title="Attack Type",
            yaxis_title="Number of Flows",
            xaxis_tickangle=-45,
            height=500,
            width=1000,
            showlegend=False
        )
        
        # Save to HTML
        fig.write_html(os.path.join(self.output_dir, 'attack_distribution.html'))
    
    def create_flow_timeline(self, df):
        """
        Create scatter plot showing traffic patterns over time.
        Reveals temporal patterns in attacks and normal traffic.
        """
        if 'flow_duration' not in df.columns or 'total_bytes' not in df.columns:
            return
        
        # Further sample if still too large for smooth visualization
        if len(df) > 1000:
            df = df.sample(1000)
        
        fig = go.Figure()
        
        # Color by attack type if available
        if 'attack_type' in df.columns:
            # Create trace for each attack type
            for attack_type in df['attack_type'].unique():
                mask = df['attack_type'] == attack_type
                
                # Determine color
                color = 'green' if attack_type == 'BENIGN' else 'red'
                
                fig.add_trace(go.Scatter(
                    x=df[mask].index,
                    y=df[mask]['total_bytes'],
                    mode='markers',
                    name=str(attack_type),
                    marker=dict(
                        size=np.log1p(df[mask]['total_packets']) * 2,  # Size by packet count
                        color=color,
                        opacity=0.6,
                        line=dict(width=0)
                    ),
                    hovertemplate='Flow %{x}<br>Bytes: %{y}<br>Type: ' + str(attack_type) + '<extra></extra>'
                ))
        else:
            # Single trace if no attack labels
            fig.add_trace(go.Scatter(
                x=df.index,
                y=df['total_bytes'],
                mode='markers',
                marker=dict(size=5, color='blue', opacity=0.6)
            ))
        
        fig.update_layout(
            title="Flow Timeline (Sample)",
            xaxis_title="Flow Index",
            yaxis_title="Total Bytes",
            yaxis_type="log",  # Log scale for better visibility
            height=500,
            width=1200,
            hovermode='closest'
        )
        
        # Save to HTML
        fig.write_html(os.path.join(self.output_dir, 'flow_timeline.html'))
    
    def create_port_heatmap(self, df):
        """
        Create heatmap showing communication patterns between ports.
        Helps identify service interactions and potential scanning.
        """
        if 'src_port' not in df.columns or 'dst_port' not in df.columns:
            return
        
        # Get top ports by frequency
        top_src = df['src_port'].value_counts().head(15).index
        top_dst = df['dst_port'].value_counts().head(15).index
        
        # Filter for top ports
        df_filtered = df[df['src_port'].isin(top_src) & df['dst_port'].isin(top_dst)]
        
        # Create communication matrix
        matrix = pd.crosstab(df_filtered['src_port'], df_filtered['dst_port'])
        
        # Create heatmap
        fig = go.Figure(data=go.Heatmap(
            z=matrix.values,
            x=[str(int(p)) for p in matrix.columns],
            y=[str(int(p)) for p in matrix.index],
            colorscale='Viridis',
            hoverongaps=False,
            hovertemplate='Src Port: %{y}<br>Dst Port: %{x}<br>Count: %{z}<extra></extra>'
        ))
        
        fig.update_layout(
            title="Port Communication Heatmap (Top Ports)",
            xaxis_title="Destination Port",
            yaxis_title="Source Port",
            height=600,
            width=800
        )
        
        # Save to HTML
        fig.write_html(os.path.join(self.output_dir, 'port_heatmap.html'))
    
    def create_feature_correlation(self, df):
        """
        Create correlation matrix of top features.
        Identifies redundant features and relationships.
        """
        # Select numeric columns (excluding metadata)
        exclude_cols = {'flow_id', 'label', 'attack_type', 'label_confidence'}
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        numeric_cols = [col for col in numeric_cols if col not in exclude_cols][:15]
        
        if len(numeric_cols) < 2:
            return
        
        # Calculate correlation matrix
        corr_matrix = df[numeric_cols].corr()
        
        # Create heatmap
        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.columns,
            colorscale='RdBu',
            zmid=0,  # Center colorscale at 0
            colorbar=dict(title="Correlation"),
            hovertemplate='%{x}<br>%{y}<br>Correlation: %{z:.2f}<extra></extra>'
        ))
        
        fig.update_layout(
            title="Feature Correlation Matrix (Top Features)",
            height=700,
            width=800
        )
        
        # Save to HTML
        fig.write_html(os.path.join(self.output_dir, 'feature_correlation.html'))
    
    def create_cicids_feature_analysis(self, df):
        """
        Create CICIDS-specific visualizations for forward/backward features.
        Shows directional traffic patterns important for CICIDS.
        """
        # Check for CICIDS features
        fwd_cols = [col for col in df.columns if 'fwd' in col.lower()]
        bwd_cols = [col for col in df.columns if 'bwd' in col.lower()]
        
        if not fwd_cols or not bwd_cols:
            return
        
        # Create subplot figure
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Forward vs Backward Packets', 'Forward vs Backward Bytes',
                          'Active vs Idle Time', 'Flag Distribution'),
            specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
                   [{'type': 'bar'}, {'type': 'bar'}]]
        )
        
        # Forward vs Backward packets
        if 'fwd_packets' in df.columns and 'bwd_packets' in df.columns:
            fig.add_trace(
                go.Scatter(x=df['fwd_packets'], y=df['bwd_packets'],
                          mode='markers', marker=dict(size=3),
                          name='Fwd vs Bwd Packets'),
                row=1, col=1
            )
        
        # Forward vs Backward bytes
        if 'fwd_bytes' in df.columns and 'bwd_bytes' in df.columns:
            fig.add_trace(
                go.Scatter(x=df['fwd_bytes'], y=df['bwd_bytes'],
                          mode='markers', marker=dict(size=3),
                          name='Fwd vs Bwd Bytes'),
                row=1, col=2
            )
        
        # Active vs Idle statistics
        if 'active_mean' in df.columns and 'idle_mean' in df.columns:
            active_idle_data = pd.DataFrame({
                'Active': df['active_mean'].mean(),
                'Idle': df['idle_mean'].mean()
            }, index=[0])
            
            fig.add_trace(
                go.Bar(x=['Active', 'Idle'],
                      y=[active_idle_data['Active'].values[0], active_idle_data['Idle'].values[0]],
                      name='Active/Idle'),
                row=2, col=1
            )
        
        # Flag distribution
        flag_cols = ['syn_count', 'ack_count', 'fin_count', 'rst_count', 'psh_count']
        flag_means = []
        flag_names = []
        for col in flag_cols:
            if col in df.columns:
                flag_means.append(df[col].mean())
                flag_names.append(col.replace('_count', '').upper())
        
        if flag_means:
            fig.add_trace(
                go.Bar(x=flag_names, y=flag_means, name='Flags'),
                row=2, col=2
            )
        
        fig.update_layout(height=800, width=1200, title="CICIDS Feature Analysis")
        
        # Save to HTML
        fig.write_html(os.path.join(self.output_dir, 'cicids_features.html'))
    
    def create_model_comparison(self):
        """
        Create bar chart comparing ML model performance.
        Shows which algorithms work best for this data.
        """
        if not self.ml_results:
            return
        
        # Extract model names and accuracies
        models = list(self.ml_results.keys())
        accuracies = [self.ml_results[m]['accuracy'] for m in models]
        
        # Create bar chart
        fig = go.Figure(data=[go.Bar(
            x=models,
            y=accuracies,
            text=[f"{acc:.4f}" for acc in accuracies],
            textposition='auto',
            marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96A6A6'][:len(models)],
            hovertemplate='%{x}<br>Accuracy: %{y:.4f}<extra></extra>'
        )])
        
        fig.update_layout(
            title="Model Performance Comparison",
            xaxis_title="Model",
            yaxis_title="Accuracy",
            yaxis_range=[0, 1],
            height=400,
            width=600
        )
        
        # Save to HTML
        fig.write_html(os.path.join(self.output_dir, 'model_comparison.html'))
    
    def create_dashboard(self, sample_df):
        """
        Create integrated HTML dashboard combining all visualizations.
        Provides single-page overview of analysis results.
        """
        # Calculate summary statistics
        total_flows = self.get_total_flow_count()
        attack_flows = sample_df[sample_df.get('label', 0) > 0].shape[0] if 'label' in sample_df.columns else 0
        attack_pct = (attack_flows / len(sample_df) * 100) if len(sample_df) > 0 else 0
        best_accuracy = max(r['accuracy'] for r in self.ml_results.values()) if self.ml_results else 0
        
        # Create dashboard HTML
        dashboard_html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Network Analysis Dashboard</title>
            <style>
                body {{ 
                    font-family: 'Segoe UI', Arial, sans-serif; 
                    margin: 20px; 
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    min-height: 100vh;
                }}
                .container {{
                    max-width: 1400px;
                    margin: 0 auto;
                    background: rgba(255,255,255,0.95);
                    border-radius: 20px;
                    padding: 30px;
                    box-shadow: 0 20px 60px rgba(0,0,0,0.3);
                }}
                h1 {{ 
                    color: #2d3748; 
                    border-bottom: 3px solid #667eea; 
                    padding-bottom: 15px;
                    margin-bottom: 30px;
                    font-size: 28px;
                }}
                .stats {{ 
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    padding: 25px;
                    border-radius: 15px;
                    margin-bottom: 30px;
                    color: white;
                    box-shadow: 0 10px 30px rgba(0,0,0,0.2);
                }}
                .stats h2 {{
                    margin-top: 0;
                    font-size: 20px;
                    opacity: 0.9;
                }}
                .stat-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                    gap: 20px;
                    margin-top: 15px;
                }}
                .stat-item {{
                    background: rgba(255,255,255,0.1);
                    padding: 15px;
                    border-radius: 10px;
                    backdrop-filter: blur(10px);
                }}
                .stat-value {{
                    font-size: 24px;
                    font-weight: bold;
                    margin-bottom: 5px;
                }}
                .stat-label {{
                    font-size: 12px;
                    opacity: 0.8;
                    text-transform: uppercase;
                }}
                .warning {{ 
                    background: #fed7d7;
                    color: #742a2a;
                    padding: 15px;
                    border-radius: 10px;
                    margin-bottom: 20px;
                    border-left: 4px solid #fc8181;
                }}
                .grid {{ 
                    display: grid;
                    grid-template-columns: repeat(2, 1fr);
                    gap: 25px;
                    margin-top: 20px;
                }}
                .viz-frame {{ 
                    background: white;
                    border-radius: 12px;
                    padding: 15px;
                    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
                    transition: transform 0.3s ease;
                }}
                .viz-frame:hover {{
                    transform: translateY(-5px);
                    box-shadow: 0 8px 25px rgba(0,0,0,0.15);
                }}
                iframe {{ 
                    width: 100%;
                    height: 400px;
                    border: none;
                    border-radius: 8px;
                }}
                .full-width {{ 
                    grid-column: span 2;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>CICIDS2017 Network Analysis Dashboard</h1>
                
                <div class="warning">
                    <strong>Note:</strong> Visualizations based on sampled data ({len(sample_df):,} flows).
                    Full dataset contains {total_flows:,} flows.
                </div>
                
                <div class="stats">
                    <h2>Summary Statistics</h2>
                    <div class="stat-grid">
                        <div class="stat-item">
                            <div class="stat-value">{total_flows:,}</div>
                            <div class="stat-label">Total Flows Processed</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-value">{attack_pct:.1f}%</div>
                            <div class="stat-label">Attack Traffic (Sample)</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-value">{best_accuracy:.2%}</div>
                            <div class="stat-label">Best Model Accuracy</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-value">{psutil.virtual_memory().percent:.1f}%</div>
                            <div class="stat-label">Memory Usage</div>
                        </div>
                    </div>
                </div>
                
                <div class="grid">
                    <div class="viz-frame">
                        <h3>Protocol Distribution</h3>
                        <iframe src="protocol_distribution.html"></iframe>
                    </div>
                    <div class="viz-frame">
                        <h3>Model Performance</h3>
                        <iframe src="model_comparison.html"></iframe>
                    </div>
                    <div class="viz-frame full-width">
                        <h3>Attack Type Distribution</h3>
                        <iframe src="attack_distribution.html"></iframe>
                    </div>
                    <div class="viz-frame full-width">
                        <h3>Flow Timeline Analysis</h3>
                        <iframe src="flow_timeline.html"></iframe>
                    </div>
                    <div class="viz-frame">
                        <h3>Port Communication Patterns</h3>
                        <iframe src="port_heatmap.html"></iframe>
                    </div>
                    <div class="viz-frame">
                        <h3>Feature Correlations</h3>
                        <iframe src="feature_correlation.html"></iframe>
                    </div>
                    <div class="viz-frame full-width">
                        <h3>CICIDS Feature Analysis</h3>
                        <iframe src="cicids_features.html"></iframe>
                    </div>
                </div>
                
                <p style="text-align: center; color: #718096; margin-top: 30px; font-size: 14px;">
                    Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | 
                    Pipeline: Memory-Optimized Network Analysis V2.0
                </p>
            </div>
        </body>
        </html>
        """
        
        # Save dashboard
        with open(os.path.join(self.output_dir, 'dashboard.html'), 'w') as f:
            f.write(dashboard_html)
        
        print("\nDashboard created: dashboard.html")
        print("  Open in browser to view all visualizations")
    
    def get_total_flow_count(self):
        """
        Get total number of flows from Parquet files.
        Counts rows without loading data into memory.
        
        Returns:
            int: Total flow count
        """
        total = 0
        import glob
        feature_files = glob.glob(os.path.join(self.features_dir, 'batch_*.parquet'))
        
        for file in feature_files:
            # Get row count from Parquet metadata
            df = pd.read_parquet(file, columns=['flow_id'])
            total += len(df)
            del df
            gc.collect()
        
        return total

# ### Step 10: Export and Reporting

In [11]:
# Cell 10: Result Exporter - Saving Analysis Outputs
"""
PURPOSE: Export all analysis results in various formats for use and sharing
This class handles saving features, models, and reports to disk.

WHAT THIS CELL DOES:
1. Exports extracted features to CSV format (in chunks)
2. Saves trained ML models for reuse
3. Generates comprehensive text report
4. Creates metadata manifest for tracking
5. Handles cleanup of temporary files

OUTPUT FILES CREATED:
- features_chunk_*.csv: Feature data split into manageable files
- *.pkl: Serialized ML models
- scaler.pkl: Feature scaler for inference
- analysis_report.txt: Detailed text report
- manifest.json: Metadata about the analysis
- dashboard.html: Interactive visualization dashboard

WHY CHUNKED EXPORT:
- CSV files have size limits
- Easier to transfer and share
- Can be processed separately
- Prevents memory overflow during export

REPORT CONTENTS:
- Processing statistics
- Label distribution  
- Feature importance
- Model performance
- Memory usage

IMPORTANT CHANGES:
- Reading from Parquet instead of HDF5
- Exports feature scaler for inference
- Better handling of large exports
- Includes CICIDS-specific statistics
"""

class MemoryOptimizedResultExporter:
    def __init__(self, output_dir):
        """
        Initialize exporter with output directory.
        
        Args:
            output_dir: Directory for saving all outputs
        """
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
    
    def export_features_chunked(self, features_dir, chunk_size=100000):
        """
        Export features from Parquet to CSV in manageable chunks.
        Large datasets are split across multiple CSV files.
        
        Args:
            features_dir: Directory containing feature Parquet files
            chunk_size: Maximum rows per CSV file
            
        Creates:
            features_chunk_000.csv, features_chunk_001.csv, etc.
        """
        print("\n" + "="*60)
        print("EXPORTING FEATURES IN CHUNKS")
        print("="*60)
        print(f"Exporting to CSV with max {chunk_size:,} rows per file")
        
        chunk_counter = 0
        total_rows = 0
        file_list = []
        
        import glob
        feature_files = sorted(glob.glob(os.path.join(features_dir, 'batch_*.parquet')))
        
        for file in tqdm(feature_files, desc="Exporting chunks"):
            # Load batch from Parquet
            batch = pd.read_parquet(file)
            
            # Split batch if it's too large
            for start_idx in range(0, len(batch), chunk_size):
                end_idx = min(start_idx + chunk_size, len(batch))
                chunk = batch.iloc[start_idx:end_idx]
                
                # Save to CSV
                chunk_file = os.path.join(
                    self.output_dir, 
                    f'features_chunk_{chunk_counter:03d}.csv'
                )
                
                # Save with compression to reduce file size
                chunk.to_csv(chunk_file + '.gz', index=False, compression='gzip')
                
                # Track file info
                file_info = {
                    'filename': os.path.basename(chunk_file) + '.gz',
                    'rows': len(chunk),
                    'size_mb': os.path.getsize(chunk_file + '.gz') / (1024*1024)
                }
                file_list.append(file_info)
                
                chunk_counter += 1
                total_rows += len(chunk)
            
            # Clean up
            del batch
            gc.collect()
        
        print(f"\n✓ Exported {total_rows:,} flows in {chunk_counter} chunks")
        
        # Create manifest file with export metadata
        manifest_file = os.path.join(self.output_dir, 'manifest.json')
        manifest = {
            'export_date': datetime.now().isoformat(),
            'total_rows': total_rows,
            'total_chunks': chunk_counter,
            'chunk_pattern': 'features_chunk_*.csv.gz',
            'compression': 'gzip',
            'files': file_list,
            'source': {
                'pcap': Config.PCAP_FILE,
                'analysis_mode': Config.ANALYSIS_MODE,
                'chunk_size': Config.CHUNK_SIZE,
                'storage_format': Config.STORAGE_FORMAT
            }
        }
        
        with open(manifest_file, 'w') as f:
            json.dump(manifest, f, indent=2)
        
        print(f"✓ Manifest saved to: {manifest_file}")
    
    def export_models(self, models, scaler=None, selected_features=None):
        """
        Save trained ML models, scaler, and selected features for future use.
        Models are serialized using pickle format.
        
        Args:
            models: Dictionary of trained model objects
            scaler: Feature scaler object
            selected_features: List of selected feature names
        """
        print("\n" + "="*60)
        print("EXPORTING ML MODELS")
        print("="*60)
        
        # Export models
        for model_name, model in models.items():
            if model is not None:
                # Create filename
                model_path = os.path.join(self.output_dir, f'{model_name}.pkl')
                
                # Serialize model
                with open(model_path, 'wb') as f:
                    pickle.dump(model, f)
                
                # Get file size
                size_mb = os.path.getsize(model_path) / (1024*1024)
                print(f"✓ Saved {model_name}: {size_mb:.2f} MB")
        
        # Export scaler (important for inference)
        if scaler is not None:
            scaler_path = os.path.join(self.output_dir, 'scaler.pkl')
            with open(scaler_path, 'wb') as f:
                pickle.dump(scaler, f)
            print(f"✓ Saved feature scaler")
        
        # Export selected features (needed for inference)
        if selected_features is not None:
            features_path = os.path.join(self.output_dir, 'selected_features.pkl')
            with open(features_path, 'wb') as f:
                pickle.dump(selected_features, f)
            print(f"✓ Saved {len(selected_features)} selected feature names")
    
    def generate_report(self, features_dir, ml_results, selected_features):
        """
        Generate comprehensive text report of analysis results.
        Provides human-readable summary of entire pipeline.
        
        Args:
            features_dir: Directory containing feature files
            ml_results: ML performance results
            selected_features: List of selected features
        """
        report_path = os.path.join(self.output_dir, 'analysis_report.txt')
        
        # Collect statistics without loading all data
        stats = self.collect_statistics(features_dir)
        
        with open(report_path, 'w') as f:
            # Header
            f.write("="*70 + "\n")
            f.write("NETWORK PACKET ANALYSIS REPORT\n")
            f.write("Memory-Optimized Pipeline V2.0 for CICIDS2017\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*70 + "\n\n")
            
            # System Information
            f.write("SYSTEM INFORMATION\n")
            f.write("-"*40 + "\n")
            f.write(f"Total RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB\n")
            f.write(f"CPU: {psutil.cpu_count()} cores\n")
            f.write(f"Storage Format: {Config.STORAGE_FORMAT}\n")
            f.write("\n")
            
            # Input Information
            f.write("INPUT CONFIGURATION\n")
            f.write("-"*40 + "\n")
            f.write(f"PCAP File: {os.path.basename(Config.PCAP_FILE)}\n")
            if os.path.exists(Config.PCAP_FILE):
                f.write(f"File Size: {os.path.getsize(Config.PCAP_FILE) / (1024**3):.2f} GB\n")
            f.write(f"Analysis Mode: {Config.ANALYSIS_MODE}\n")
            f.write(f"Deep Inspection: {Config.DEEP_INSPECTION}\n")
            f.write("\n")
            
            # Processing Statistics
            f.write("PROCESSING STATISTICS\n")
            f.write("-"*40 + "\n")
            f.write(f"Total Flows: {stats['total_flows']:,}\n")
            f.write(f"Processing Strategy: {'Disk-based' if Config.USE_DISK_CACHE else 'Memory-based'}\n")
            f.write(f"Max Memory Setting: {Config.MAX_MEMORY_GB:.1f} GB\n")
            f.write(f"Chunk Size: {Config.CHUNK_SIZE:,} packets\n")
            f.write(f"Batch Size: {Config.BATCH_SIZE:,} flows\n")
            f.write(f"Max Flows in Memory: {Config.MAX_FLOWS_IN_MEMORY:,}\n")
            f.write(f"Max Packets per Flow: {Config.MAX_PACKETS_PER_FLOW:,}\n")
            f.write("\n")
            
            # Label Distribution
            if stats.get('label_distribution'):
                f.write("ATTACK TYPE DISTRIBUTION\n")
                f.write("-"*40 + "\n")
                total = sum(stats['label_distribution'].values())
                for label, count in sorted(stats['label_distribution'].items(), 
                                          key=lambda x: x[1], reverse=True):
                    pct = count / total * 100 if total > 0 else 0
                    f.write(f"  {label:30s}: {count:10,} ({pct:5.1f}%)\n")
                f.write(f"\nTotal Labeled Flows: {total:,}\n")
                
                # CICIDS-specific insights
                benign_ratio = stats['label_distribution'].get('BENIGN', 0) / total
                attack_ratio = 1 - benign_ratio
                f.write(f"Attack Traffic Ratio: {attack_ratio:.2%}\n")
                f.write("\n")
            
            # Feature Selection
            if selected_features:
                f.write("FEATURE SELECTION\n")
                f.write("-"*40 + "\n")
                f.write(f"Features Analyzed: {stats.get('total_features', 'Unknown')}\n")
                f.write(f"Features Selected: {len(selected_features)}\n")
                
                # Categorize CICIDS features
                cicids_features = [f for f in selected_features if any(x in f.lower() 
                                  for x in ['fwd', 'bwd', 'active', 'idle', 'iat'])]
                f.write(f"CICIDS-specific Features: {len(cicids_features)}\n")
                
                f.write("\nTop 10 Selected Features:\n")
                for i, feature in enumerate(selected_features[:10], 1):
                    f.write(f"  {i:2d}. {feature}\n")
                f.write("\n")
            
            # ML Results
            if ml_results:
                f.write("MACHINE LEARNING RESULTS\n")
                f.write("-"*40 + "\n")
                
                # Find best model
                best_model = max(ml_results.items(), key=lambda x: x[1]['accuracy'])
                
                for model_name, results in ml_results.items():
                    is_best = model_name == best_model[0]
                    marker = " ⭐" if is_best else ""
                    
                    f.write(f"\n{model_name.upper()}{marker}:\n")
                    f.write(f"  Test Accuracy: {results['accuracy']:.4f}\n")
                    
                    if 'test_samples' in results:
                        f.write(f"  Test Samples: {results['test_samples']:,}\n")
                    
                    if 'unique_predictions' in results:
                        f.write(f"  Unique Predictions: {results['unique_predictions']}\n")
                
                f.write(f"\nBest Model: {best_model[0]} (Accuracy: {best_model[1]['accuracy']:.4f})\n")
                f.write("\n")
            
            # Memory Usage
            mem = psutil.virtual_memory()
            f.write("MEMORY USAGE\n")
            f.write("-"*40 + "\n")
            f.write(f"Current Memory: {mem.percent:.1f}%\n")
            f.write(f"Available: {mem.available / (1024**3):.2f} GB\n")
            f.write(f"Used: {mem.used / (1024**3):.2f} GB\n")
            f.write(f"Total System: {mem.total / (1024**3):.2f} GB\n")
            f.write("\n")
            
            # Recommendations
            f.write("ANALYSIS INSIGHTS & RECOMMENDATIONS\n")
            f.write("-"*40 + "\n")
            
            # Check for class imbalance
            if stats.get('label_distribution'):
                benign_ratio = stats['label_distribution'].get('BENIGN', 0) / stats['total_flows']
                if benign_ratio > 0.9:
                    f.write("⚠️  High class imbalance detected (>90% BENIGN)\n")
                    f.write("   Consider: Oversampling attacks or using weighted loss\n\n")
                elif benign_ratio > 0.8:
                    f.write("✓ Moderate class imbalance (80-90% BENIGN)\n")
                    f.write("   This is typical for CICIDS2017 dataset\n\n")
            
            # Check accuracy
            if ml_results:
                avg_accuracy = np.mean([r['accuracy'] for r in ml_results.values()])
                if avg_accuracy < 0.8:
                    f.write("⚠️  Low average accuracy (<80%)\n")
                    f.write("   Consider: Increasing features, tuning hyperparameters\n\n")
                elif avg_accuracy > 0.95:
                    f.write("✓ Excellent model performance (>95%)\n")
                    f.write("   Ready for deployment consideration\n\n")
                else:
                    f.write("✓ Good model performance (80-95%)\n")
                    f.write("   Consider ensemble methods for improvement\n\n")
            
            # Storage recommendations
            f.write("Storage Format: ")
            if Config.STORAGE_FORMAT == 'parquet':
                f.write("✓ Using Parquet (optimal for Windows)\n\n")
            else:
                f.write("Consider using Parquet for better performance\n\n")
            
            # Footer
            f.write("\n" + "="*70 + "\n")
            f.write("END OF REPORT\n")
            f.write("="*70 + "\n")
        
        print(f"\n✓ Report saved to: {report_path}")
    
    def collect_statistics(self, features_dir):
        """
        Collect statistics from Parquet files without loading all data.
        Uses metadata and sampling for efficiency.
        
        Args:
            features_dir: Directory containing feature files
            
        Returns:
            dict: Statistics dictionary
        """
        stats = {
            'total_flows': 0,
            'total_features': 0,
            'label_distribution': Counter()
        }
        
        import glob
        feature_files = sorted(glob.glob(os.path.join(features_dir, 'batch_*.parquet')))
        
        for file in feature_files:
            # Get row count
            df_sample = pd.read_parquet(file, columns=['flow_id'])
            stats['total_flows'] += len(df_sample)
            
            # Get feature count from first file
            if stats['total_features'] == 0:
                df_first = pd.read_parquet(file)
                stats['total_features'] = len(df_first.columns)
            
            # Sample for label distribution
            if 'attack_type' in pd.read_parquet(file, columns=[]).columns:
                df_labels = pd.read_parquet(file, columns=['attack_type'])
                stats['label_distribution'].update(
                    df_labels['attack_type'].value_counts().to_dict()
                )
            
            del df_sample
            gc.collect()
        
        return stats
    
    def export_all(self, features_dir, models, ml_results, selected_features, scaler=None):
        """
        Main export function - coordinates all export operations.
        
        Args:
            features_dir: Directory containing feature files
            models: Trained ML models
            ml_results: Model performance results
            selected_features: Selected feature names
            scaler: Feature scaler
        """
        print("\n" + "="*70)
        print("EXPORTING ALL RESULTS")
        print("="*70)
        
        # Export features if requested
        if Config.ML_EXPORT:
            self.export_features_chunked(features_dir)
        
        # Export models, scaler, and selected features
        if models:
            self.export_models(models, scaler, selected_features)
        
        # Generate report
        self.generate_report(features_dir, ml_results, selected_features)
        
        print(f"\n✓ All results exported to: {self.output_dir}")
        print("  Files created:")
        print("    - features_chunk_*.csv.gz (data)")
        print("    - *.pkl (models)")
        print("    - scaler.pkl (feature scaler)")
        print("    - selected_features.pkl (feature names)")
        print("    - analysis_report.txt (summary)")
        print("    - manifest.json (metadata)")
        print("    - dashboard.html (visualizations)")

# ### Step 11: Main Execution Function

In [12]:
# Cell 11: Main Pipeline - Complete Analysis Orchestration
"""
PURPOSE: Orchestrate the entire analysis pipeline from start to finish
This is the main execution function that coordinates all components.

WHAT THIS CELL DOES:
1. Validates configuration and checks system resources
2. Extracts features from PCAP file
3. Matches with CICIDS labels if available
4. Analyzes and selects best features
5. Trains ML models incrementally
6. Generates visualizations
7. Exports all results
8. Cleans up temporary files

PIPELINE FLOW:
Config → Extract → Label → Analyze → Train → Visualize → Export → Cleanup

ERROR HANDLING:
- Saves partial results on failure
- Cleans up temporary files
- Provides detailed error messages
- Suggests fixes for common issues

MEMORY MANAGEMENT:
- Monitors memory throughout execution
- Switches strategies based on usage
- Cleans up after each phase
- Reports memory statistics

IMPORTANT CHANGES:
- Using Parquet throughout pipeline
- Better error recovery with checkpoints
- Parallel processing where applicable
- Optimized for AMD Ryzen CPU
"""

def run_memory_optimized_pipeline():
    """
    Main execution function for the complete analysis pipeline.
    Orchestrates all components while managing memory efficiently.
    
    Returns:
        dict: Results dictionary with paths and metrics
    """
    print("\n" + "="*70)
    print("STARTING MEMORY-OPTIMIZED NETWORK ANALYSIS PIPELINE V2.0")
    print("="*70)
    print(f"Pipeline Version: 2.0 (Parquet-based, Windows-optimized)")
    print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"System: {psutil.virtual_memory().total / (1024**3):.1f}GB RAM, {psutil.cpu_count()} CPU cores")
    
    # Check if ML should be skipped
    if Config.SKIP_ML:
        print("\nML TRAINING WILL BE SKIPPED (Test Mode)")
    
    # Check configuration and memory
    strategy = Config.check_memory_requirements()
    
    if strategy is False:
        print("Error: Invalid PCAP file or configuration")
        return None
    
    print(f"\nProcessing Strategy: {strategy}")
    print(f"Memory Limit: {Config.MAX_MEMORY_GB:.1f} GB")
    print(f"Storage Format: {Config.STORAGE_FORMAT}")
    
    # Initialize results tracking
    results = {
        'start_time': datetime.now(),
        'strategy': strategy,
        'temp_dirs': []
    }
    
    try:
        # ========== STEP 1: FEATURE EXTRACTION ==========
        print("\n" + "="*70)
        print("STEP 1: STREAMING FEATURE EXTRACTION")
        print("="*70)
        print("Extracting features from network packets...")
        print("Using Parquet for reliable storage (no file locking)")
        
        pipeline = MemoryOptimizedFeaturePipeline()
        features_dir = pipeline.extract_all_features(
            Config.PCAP_FILE,
            mode=Config.ANALYSIS_MODE
        )
        
        if not features_dir:
            print("Error: No features extracted")
            return None
        
        results['temp_dirs'].append(features_dir)
        results['features_dir'] = features_dir
        
        # Memory checkpoint
        monitor_memory("After feature extraction")
        
        # ========== STEP 2: LABEL MATCHING ==========
        if Config.USE_CICIDS_LABELS and Config.LABEL_FILES:
            print("\n" + "="*70)
            print("STEP 2: INCREMENTAL LABEL MATCHING")
            print("="*70)
            print("Matching flows with CICIDS ground truth labels...")
            
            label_matcher = MemoryOptimizedLabelMatcher()
            labeled_dir = label_matcher.process_features_with_labels(
                features_dir, Config.LABEL_FILES
            )
            
            features_dir = labeled_dir
            results['temp_dirs'].append(labeled_dir)
            results['features_dir'] = labeled_dir
            
            monitor_memory("After label matching")
        else:
            print("\nStep 2: Skipping label matching (no labels provided)")
        
        # ========== STEP 3: FEATURE ANALYSIS ==========
        print("\n" + "="*70)
        print("STEP 3: INCREMENTAL FEATURE ANALYSIS")
        print("="*70)
        print("Analyzing feature importance and selecting best features...")
        
        analyzer = MemoryOptimizedFeatureAnalyzer()
        selected_features = analyzer.analyze_features_incrementally(features_dir)
        results['selected_features'] = selected_features
        
        # Get feature insights
        insights = analyzer.get_feature_insights()
        results['feature_insights'] = insights
        
        monitor_memory("After feature analysis")
        
        # ========== STEP 4: MACHINE LEARNING ==========
        if not Config.SKIP_ML:
            print("\n" + "="*70)
            print("STEP 4: INCREMENTAL MACHINE LEARNING")
            print("="*70)
            print("Training ML models with incremental learning...")
            print(f"Models to train: {Config.SELECTED_MODELS}")
            
            ml_pipeline = MemoryOptimizedMLPipeline()
            models, ml_results = ml_pipeline.train_models_incrementally(
                features_dir, selected_features
            )
            
            results['models'] = models
            results['ml_results'] = ml_results
            results['scaler'] = ml_pipeline.scaler
            
            monitor_memory("After ML training")
        else:
            print("\n" + "="*70)
            print("STEP 4: SKIPPING ML TRAINING (Test Mode)")
            print("="*70)
            print("ML training skipped for pipeline validation")
            print("Feature extraction and labeling completed successfully")
            models = {}
            ml_results = {}
            results['models'] = models
            results['ml_results'] = ml_results
            results['scaler'] = None
        
        # ========== STEP 5: VISUALIZATION ==========
        if Config.GENERATE_VISUALS and not Config.SKIP_ML:
            print("\n" + "="*70)
            print("STEP 5: SAMPLED VISUALIZATIONS")
            print("="*70)
            print("Creating interactive visualizations...")
            
            visualizer = MemoryOptimizedVisualizer(
                features_dir, ml_results, Config.OUTPUT_DIR
            )
            visualizer.create_all_visualizations()
            
            monitor_memory("After visualization")
        else:
            if Config.SKIP_ML:
                print("\nStep 5: Skipping visualizations (ML was skipped)")
            else:
                print("\nStep 5: Skipping visualizations (disabled)")
        
        # ========== STEP 6: EXPORT RESULTS ==========
        print("\n" + "="*70)
        print("STEP 6: CHUNKED EXPORT")
        print("="*70)
        print("Exporting all results to disk...")
        
        exporter = MemoryOptimizedResultExporter(Config.OUTPUT_DIR)
        exporter.export_all(
            features_dir, 
            models, 
            ml_results, 
            selected_features,
            results.get('scaler')
        )
        
        monitor_memory("After export")
        
        # ========== STEP 7: CLEANUP ==========
        if Config.CLEANUP_TEMP:
            print("\n" + "="*70)
            print("STEP 7: CLEANUP")
            print("="*70)
            print("Removing temporary files...")
            
            import shutil
            for temp_dir in results['temp_dirs']:
                if os.path.exists(temp_dir):
                    try:
                        shutil.rmtree(temp_dir)
                        print(f"  Removed: {os.path.basename(temp_dir)}")
                    except Exception as e:
                        print(f"  Failed to remove {temp_dir}: {e}")
        
        # ========== FINAL SUMMARY ==========
        print("\n" + "="*70)
        print("PIPELINE COMPLETE")
        print("="*70)
        
        # Calculate execution time
        results['end_time'] = datetime.now()
        duration = (results['end_time'] - results['start_time']).total_seconds() / 60
        
        print(f"Analysis completed successfully")
        print(f"  Duration: {duration:.1f} minutes")
        print(f"  Output Directory: {Config.OUTPUT_DIR}")
        print(f"  Storage Format: {Config.STORAGE_FORMAT}")
        
        # Memory summary
        mem = psutil.virtual_memory()
        print(f"\nMemory Summary:")
        print(f"  Peak Usage: ~{Config.MAX_MEMORY_GB:.1f} GB (estimated)")
        print(f"  Current Usage: {mem.percent:.1f}%")
        print(f"  Strategy: {strategy}")
        
        # ML summary
        if ml_results and not Config.SKIP_ML:
            best_model = max(ml_results.items(), key=lambda x: x[1]['accuracy'])
            print(f"\nBest Model: {best_model[0]}")
            print(f"  Accuracy: {best_model[1]['accuracy']:.4f}")
        elif Config.SKIP_ML:
            print(f"\nML Training: Skipped (Test Mode)")
            print(f"  Features extracted: {len(selected_features)}")
        
        # Create success summary
        results['summary'] = {
            'success': True,
            'duration_minutes': duration,
            'strategy': strategy,
            'memory_limit_gb': Config.MAX_MEMORY_GB,
            'best_accuracy': max(r['accuracy'] for r in ml_results.values()) if ml_results else 0,
            'features_selected': len(selected_features),
            'output_dir': Config.OUTPUT_DIR,
            'ml_skipped': Config.SKIP_ML,
            'storage_format': Config.STORAGE_FORMAT
        }
        
        print("\n✓ Pipeline execution successful!")
        print(f"   View results in: {Config.OUTPUT_DIR}")
        if not Config.SKIP_ML:
            print(f"   Open dashboard.html for visualizations")
        
        return results
        
    except Exception as e:
        # Error handling
        print(f"\n✗ Pipeline Error: {e}")
        import traceback
        traceback.print_exc()
        
        # Try to save partial results
        print("\nAttempting to save partial results...")
        if 'features_dir' in results and results['features_dir']:
            try:
                emergency_dir = os.path.join(Config.OUTPUT_DIR, 'emergency_features')
                import shutil
                shutil.copytree(results['features_dir'], emergency_dir)
                print(f"Partial results saved to: {emergency_dir}")
            except:
                print("Could not save partial results")
        
        # Cleanup on error
        print("\nCleaning up after error...")
        import shutil
        for temp_dir in results.get('temp_dirs', []):
            if os.path.exists(temp_dir):
                try:
                    shutil.rmtree(temp_dir)
                except:
                    pass
        
        # Error summary
        results['summary'] = {
            'success': False,
            'error': str(e),
            'partial_results': 'emergency_features' if 'features_dir' in results else None
        }
        
        return results


def monitor_memory(checkpoint_name=""):
    """
    Monitor and display current memory usage.
    Helps track memory consumption throughout pipeline.
    
    Args:
        checkpoint_name: Description of current pipeline stage
    """
    mem = psutil.virtual_memory()
    print(f"\nMemory Status {checkpoint_name}:")
    print(f"   Used: {mem.used / (1024**3):.2f} GB ({mem.percent:.1f}%)")
    print(f"   Available: {mem.available / (1024**3):.2f} GB")
    
    # Warning if memory usage is high
    if mem.percent > 85:
        print("   ⚠ WARNING: High memory usage detected!")
        print("      Consider reducing chunk/batch sizes")
    elif mem.percent > 95:
        print("   ⛔ CRITICAL: Very high memory usage!")
        print("      Pipeline may fail - reduce settings immediately")


def validate_environment():
    """
    Validate that all required libraries and settings are correct.
    Run this before starting the pipeline.
    """
    print("\n" + "="*70)
    print("ENVIRONMENT VALIDATION")
    print("="*70)
    
    issues = []
    
    # Check required libraries
    try:
        import pyarrow
        print("✓ PyArrow installed")
    except:
        issues.append("PyArrow not installed (required for Parquet)")
    
    try:
        import lightgbm
        print("✓ LightGBM installed")
    except:
        print("  LightGBM not installed (optional but recommended)")
    
    # Check configuration
    if not Config.PCAP_FILE:
        issues.append("PCAP file not specified")
    elif not os.path.exists(Config.PCAP_FILE):
        issues.append(f"PCAP file not found: {Config.PCAP_FILE}")
    
    # Check directories
    if not os.path.exists(Config.OUTPUT_DIR):
        os.makedirs(Config.OUTPUT_DIR, exist_ok=True)
        print(f"✓ Created output directory: {Config.OUTPUT_DIR}")
    
    if not os.path.exists(Config.TEMP_DIR):
        os.makedirs(Config.TEMP_DIR, exist_ok=True)
        print(f"✓ Created temp directory: {Config.TEMP_DIR}")
    
    # Check memory
    if Config.MAX_MEMORY_GB > psutil.virtual_memory().available / (1024**3):
        issues.append(f"MAX_MEMORY_GB ({Config.MAX_MEMORY_GB}) exceeds available memory")
    
    # Check disk space
    disk_usage = psutil.disk_usage('/')
    if disk_usage.free < 10 * (1024**3):  # Less than 10GB free
        issues.append(f"Low disk space: {disk_usage.free / (1024**3):.1f} GB free")
    
    if issues:
        print("\n✗ Validation Failed:")
        for issue in issues:
            print(f"   - {issue}")
        return False
    else:
        print("\n✓ All checks passed!")
        return True


# ========== EXECUTION CELL ==========
print("\n" + "="*70)
print("READY TO RUN MEMORY-OPTIMIZED ANALYSIS V2.0")
print("="*70)

print("\nThis pipeline is optimized for:")
print("• Large PCAP files (10+ GB)")
print("• CICIDS2017 dataset")
print("• Windows systems (no HDF5 file locking)")
print("• AMD Ryzen CPUs (parallel processing)")
print("• 32GB RAM systems")

print("\n" + "-"*40)
print("KEY IMPROVEMENTS:")
print("-"*40)
print("✓ Parquet storage (no file locking)")
print("✓ Fixed bidirectional flow identification")
print("✓ CICIDS-specific features (Fwd/Bwd, Active/Idle)")
print("✓ LightGBM support (faster on CPU)")
print("✓ Better memory management")
print("✓ Parallel processing support")

print("\n" + "-"*40)
print("INSTRUCTIONS:")
print("-"*40)
print("1. Ensure configuration is complete (Cell 2)")
print("2. Verify PCAP file path is correct")
print("3. Check available disk space for temp files")
print("4. Validate environment: validate_environment()")
print("5. Run: results = run_memory_optimized_pipeline()")
print("6. Monitor memory with: monitor_memory()")

print("\n" + "-"*40)
print("QUICK START:")
print("-"*40)
print("# Validate environment first:")
print("validate_environment()")
print("")
print("# To run the analysis:")
print("# results = run_memory_optimized_pipeline()")
print("")
print("# To check memory anytime:")
print("# monitor_memory()")

# Display current system status
print("\n" + "-"*40)
print("CURRENT SYSTEM STATUS:")
print("-"*40)
monitor_memory()

disk_usage = psutil.disk_usage('/')
print(f"\nDisk Space:")
print(f"   Available: {disk_usage.free / (1024**3):.2f} GB")
print(f"   Used: {disk_usage.percent:.1f}%")

if Config.SKIP_ML:
    print("\n⚠ ML Training is currently DISABLED")
    print("   Uncheck 'Skip ML Training' in config to enable")

print("\n✓ Ready to start analysis!")
print("   Run: results = run_memory_optimized_pipeline()")

# Uncomment to run:
# results = run_memory_optimized_pipeline()


READY TO RUN MEMORY-OPTIMIZED ANALYSIS V2.0

This pipeline is optimized for:
• Large PCAP files (10+ GB)
• CICIDS2017 dataset
• Windows systems (no HDF5 file locking)
• AMD Ryzen CPUs (parallel processing)
• 32GB RAM systems

----------------------------------------
KEY IMPROVEMENTS:
----------------------------------------
✓ Parquet storage (no file locking)
✓ Fixed bidirectional flow identification
✓ CICIDS-specific features (Fwd/Bwd, Active/Idle)
✓ LightGBM support (faster on CPU)
✓ Better memory management
✓ Parallel processing support

----------------------------------------
INSTRUCTIONS:
----------------------------------------
1. Ensure configuration is complete (Cell 2)
2. Verify PCAP file path is correct
3. Check available disk space for temp files
4. Validate environment: validate_environment()
5. Run: results = run_memory_optimized_pipeline()
6. Monitor memory with: monitor_memory()

----------------------------------------
QUICK START:
------------------------------------

# ### Step 11: run the program

In [13]:
# Uncomment to run the feature extracotr / ML trainer:
results = run_memory_optimized_pipeline()


STARTING MEMORY-OPTIMIZED NETWORK ANALYSIS PIPELINE V2.0
Pipeline Version: 2.0 (Parquet-based, Windows-optimized)
Start Time: 2025-08-30 04:50:20
System: 31.4GB RAM, 12 CPU cores

MEMORY ASSESSMENT
PCAP Size: 10.08 GB
Total RAM: 31.37 GB
Available RAM: 13.98 GB
Max Memory Setting: 8.00 GB

RECOMMENDATION: Large file detected
- Using disk-based processing with Parquet
- Enabling flow timeout mechanism
- Using incremental learning

CPU Optimization: 12 threads available

Estimated Processing Time: 30 minutes

Processing Strategy: DISK_BASED
Memory Limit: 8.0 GB
Storage Format: parquet

STEP 1: STREAMING FEATURE EXTRACTION
Extracting features from network packets...
Using Parquet for reliable storage (no file locking)

PHASE 1: MEMORY-OPTIMIZED FLOW EXTRACTION
Extracting statistical features from network flows...
This analyzes packet timing, sizes, and patterns
Including CICIDS-specific features (Active/Idle, Fwd/Bwd)

Processing PCAP with memory optimization: M:\TUD\ATU\Masters_project\

Extracting flow features: 0it [00:00, ?it/s]


  Memory limit reached at packet 6,817,527
  Flushed 71,332 flows to ./temp_processing\flow_features\batch_0000.parquet

  Memory limit reached at packet 8,545,770
  Flushed 73,935 flows to ./temp_processing\flow_features\batch_0001.parquet

  Memory limit reached at packet 10,245,812
  Flushed 73,522 flows to ./temp_processing\flow_features\batch_0002.parquet

Flushing remaining flows...
  Flushed 60,148 flows to ./temp_processing\flow_features\batch_0003.parquet

Processed 11,709,971 packets
Total flows: 278,937
Features saved to: ./temp_processing\flow_features
✓ Flow features extracted to: ./temp_processing\flow_features

PHASE 2: MEMORY-OPTIMIZED SEMANTIC EXTRACTION
Analyzing packet payloads for malicious content...
This performs deep packet inspection and NLP analysis

Semantic analysis (memory-optimized): M:\TUD\ATU\Masters_project\datasets\CICIDS2017\Raw_PCAP\Monday-WorkingHours.pcap
Output format: Parquet (Windows-optimized)


Extracting semantic features: 0it [00:00, ?it/s]

  Flushed 75,000 semantic features to ./temp_processing\semantic_features\batch_0000.parquet
  Flushed 75,000 semantic features to ./temp_processing\semantic_features\batch_0001.parquet
  Flushed 75,000 semantic features to ./temp_processing\semantic_features\batch_0002.parquet
  Flushed 55,863 semantic features to ./temp_processing\semantic_features\batch_0003.parquet
Processed 11,709,971 packets
Semantic features saved to: ./temp_processing\semantic_features
✓ Semantic features extracted to: ./temp_processing\semantic_features

PHASE 3: MERGING FEATURES (DISK-BASED)
Combining flow and semantic features...
Merging features using disk-based operations...
This preserves memory by processing in batches


...

Testing complete: 1 batches processed

----------------------------------------
MODEL PERFORMANCE
----------------------------------------

SGD:
  Test Accuracy: 1.0000
  Test Samples: 79,725
  Unique Predictions: 1

Memory Status After ML training:
   Used: 16.10 GB (51.3%)
   Available: 15.27 GB

STEP 5: SAMPLED VISUALIZATIONS
Creating interactive visualizations...

GENERATING VISUALIZATIONS (SAMPLED DATA)
Creating visualizations with 10,000 sample points
This provides insights without loading full dataset

Loading sample of 10,000 flows for visualization...
  Loaded 4,000 flows from 4 batches


Creating visualizations:   0%|          | 0/7 [00:00<?, ?it/s]

  Created: Protocol Distribution
  Created: Attack Distribution
  Created: Flow Timeline
  Created: Port Heatmap
  Created: Feature Correlation
  Created: CICIDS Features
  Created: Model Performance

Dashboard created: dashboard.html
  Open in browser to view all visualizations

Visualizations saved to: ./analysis_output
  Open dashboard.html in browser to view all charts

Memory Status After visualization:
   Used: 16.09 GB (51.3%)
   Available: 15.28 GB

STEP 6: CHUNKED EXPORT
Exporting all results to disk...

EXPORTING ALL RESULTS

EXPORTING FEATURES IN CHUNKS
Exporting to CSV with max 100,000 rows per file


Exporting chunks:   0%|          | 0/4 [00:00<?, ?it/s]


✓ Exported 358,789 flows in 4 chunks
✓ Manifest saved to: ./analysis_output\manifest.json

EXPORTING ML MODELS
✓ Saved sgd: 0.00 MB
✓ Saved feature scaler
✓ Saved 50 selected feature names

✓ Report saved to: ./analysis_output\analysis_report.txt

✓ All results exported to: ./analysis_output
  Files created:
    - features_chunk_*.csv.gz (data)
    - *.pkl (models)
    - scaler.pkl (feature scaler)
    - selected_features.pkl (feature names)
    - analysis_report.txt (summary)
    - manifest.json (metadata)
    - dashboard.html (visualizations)

Memory Status After export:
   Used: 16.06 GB (51.2%)
   Available: 15.31 GB

STEP 7: CLEANUP
Removing temporary files...
  Removed: combined_features
  Removed: labeled_features

PIPELINE COMPLETE
Analysis completed successfully
  Duration: 211.1 minutes
  Output Directory: ./analysis_output
  Storage Format: parquet

Memory Summary:
  Peak Usage: ~8.0 GB (estimated)
  Current Usage: 51.2%
  Strategy: DISK_BASED

Best Model: sgd
  Accuracy: 1

# ### Step 12: Inference Mode - Classify New PCAP Files

In [ ]:
# Cell 12: Inference Mode - Classify New PCAP Files
"""
PURPOSE: Use trained models to classify new PCAP files
This cell loads previously trained models and applies them to analyze
new network traffic without retraining.

WHAT THIS CELL DOES:
1. Loads saved models and scaler from pickle files
2. Extracts features from new PCAP file
3. Applies trained model to classify flows
4. Outputs attack distribution and statistics
5. Optionally exports predictions to CSV

REQUIREMENTS:
- Must have completed training run with saved models
- Models saved as: sgd.pkl, lightgbm.pkl, scaler.pkl
- Feature names saved as: selected_features.pkl

USAGE:
1. First run the main pipeline to train and save models
2. Configure NEW_PCAP_FILE path
3. Run classify_new_pcap() function
"""

import pickle
import pandas as pd
import numpy as np
import os
import glob
from collections import Counter
from tqdm import tqdm

class InferenceEngine:
    """
    Engine for classifying new PCAP files using trained models
    """
    
    def __init__(self, model_dir='./analysis_output'):
        """
        Initialize inference engine with model directory
        
        Args:
            model_dir: Directory containing saved models and scaler
        """
        self.model_dir = model_dir
        self.models = {}
        self.scaler = None
        self.selected_features = None
        self.attack_mapping = {
            0: 'BENIGN',
            1: 'Bot',
            2: 'DDoS',
            3: 'DoS GoldenEye',
            4: 'DoS Hulk',
            5: 'DoS Slowhttptest',
            6: 'DoS slowloris',
            7: 'FTP-Patator',
            8: 'Heartbleed',
            9: 'Infiltration',
            10: 'PortScan',
            11: 'SSH-Patator',
            12: 'Web Attack - Brute Force',
            13: 'Web Attack - SQL Injection',
            14: 'Web Attack - XSS'
        }
    
    def load_models(self):
        """
        Load trained models and preprocessing objects from disk
        """
        print("="*60)
        print("LOADING TRAINED MODELS")
        print("="*60)
        
        # Load scaler
        scaler_path = os.path.join(self.model_dir, 'scaler.pkl')
        if os.path.exists(scaler_path):
            with open(scaler_path, 'rb') as f:
                self.scaler = pickle.load(f)
            print(f"✓ Loaded scaler from {scaler_path}")
        else:
            print(f"✗ Scaler not found at {scaler_path}")
            return False
        
        # Load selected features
        features_path = os.path.join(self.model_dir, 'selected_features.pkl')
        if os.path.exists(features_path):
            with open(features_path, 'rb') as f:
                self.selected_features = pickle.load(f)
            print(f"✓ Loaded {len(self.selected_features)} selected features")
        else:
            print(f"✗ Selected features not found at {features_path}")
            return False
        
        # Load available models
        model_files = {
            'sgd': 'sgd.pkl',
            'lightgbm': 'lightgbm.pkl',
            'xgboost': 'xgboost_incremental.pkl'
        }
        
        for model_name, filename in model_files.items():
            model_path = os.path.join(self.model_dir, filename)
            if os.path.exists(model_path):
                with open(model_path, 'rb') as f:
                    self.models[model_name] = pickle.load(f)
                print(f"✓ Loaded {model_name} model")
        
        if not self.models:
            print("✗ No models found!")
            return False
        
        print(f"\nModels ready for inference: {list(self.models.keys())}")
        return True
    
    def extract_features(self, pcap_file):
        """
        Extract features from new PCAP file
        
        Args:
            pcap_file: Path to PCAP file to analyze
            
        Returns:
            str: Path to directory containing extracted features
        """
        print("\n" + "="*60)
        print("EXTRACTING FEATURES FROM NEW PCAP")
        print("="*60)
        print(f"PCAP: {pcap_file}")
        
        # Use the existing pipeline for feature extraction
        pipeline = MemoryOptimizedFeaturePipeline()
        
        # Extract only flow features (faster, usually sufficient)
        # Change to 'combined' if you need semantic features too
        features_dir = pipeline.extract_all_features(
            pcap_file,
            mode='flow'  # or 'combined' for flow + semantic
        )
        
        return features_dir
    
    def classify_flows(self, features_dir, model_name='lightgbm'):
        """
        Classify flows using specified model
        
        Args:
            features_dir: Directory containing feature Parquet files
            model_name: Which model to use for classification
            
        Returns:
            tuple: (predictions array, flow metadata DataFrame)
        """
        if model_name not in self.models:
            print(f"✗ Model {model_name} not loaded")
            return None, None
        
        print(f"\nClassifying with {model_name}...")
        
        model = self.models[model_name]
        all_predictions = []
        all_metadata = []
        
        # Process each batch file
        feature_files = sorted(glob.glob(os.path.join(features_dir, 'batch_*.parquet')))
        
        for file in tqdm(feature_files, desc="Processing batches"):
            # Load batch
            batch = pd.read_parquet(file)
            
            # Extract available features
            available_features = [f for f in self.selected_features if f in batch.columns]
            
            if not available_features:
                print(f"✗ No matching features in {file}")
                continue
            
            # Prepare features
            X = batch[available_features].fillna(0).values.astype(np.float32)
            
            # Scale features
            X_scaled = self.scaler.transform(X)
            
            # Predict
            if model_name == 'lightgbm':
                predictions = model.predict(X_scaled, num_iteration=model.best_iteration)
            else:
                predictions = model.predict(X_scaled)
            
            all_predictions.extend(predictions)
            
            # Store metadata (flow IDs, ports, etc.)
            metadata = batch[['flow_id', 'src_port', 'dst_port', 'total_bytes', 'total_packets']].copy()
            metadata['prediction'] = predictions
            metadata['attack_type'] = [self.attack_mapping.get(int(p), 'Unknown') for p in predictions]
            all_metadata.append(metadata)
        
        # Combine results
        all_predictions = np.array(all_predictions)
        all_metadata = pd.concat(all_metadata, ignore_index=True) if all_metadata else pd.DataFrame()
        
        return all_predictions, all_metadata
    
    def analyze_results(self, predictions, metadata):
        """
        Analyze and display classification results
        
        Args:
            predictions: Array of predicted labels
            metadata: DataFrame with flow details
        """
        print("\n" + "="*60)
        print("CLASSIFICATION RESULTS")
        print("="*60)
        
        # Count attack types
        attack_counts = Counter(metadata['attack_type'].values)
        total_flows = len(predictions)
        
        print(f"\nTotal flows analyzed: {total_flows:,}")
        print("\nAttack Distribution:")
        print("-"*40)
        
        for attack_type, count in attack_counts.most_common():
            percentage = (count / total_flows) * 100
            print(f"{attack_type:30s}: {count:7,} ({percentage:5.1f}%)")
        
        # Calculate attack vs benign ratio
        benign_count = attack_counts.get('BENIGN', 0)
        attack_count = total_flows - benign_count
        attack_ratio = (attack_count / total_flows) * 100 if total_flows > 0 else 0
        
        print("\n" + "-"*40)
        print(f"Benign traffic: {benign_count:,} ({100-attack_ratio:.1f}%)")
        print(f"Attack traffic: {attack_count:,} ({attack_ratio:.1f}%)")
        
        # Find most suspicious flows
        if attack_count > 0:
            print("\n" + "-"*40)
            print("Sample Attack Flows Detected:")
            attacks = metadata[metadata['attack_type'] != 'BENIGN'].head(10)
            for _, flow in attacks.iterrows():
                print(f"  {flow['attack_type']}: Port {flow['dst_port']}, "
                      f"{flow['total_bytes']} bytes, {flow['total_packets']} packets")
        
        return attack_counts
    
    def export_results(self, metadata, output_file='predictions.csv'):
        """
        Export classification results to CSV
        
        Args:
            metadata: DataFrame with predictions
            output_file: Output CSV filename
        """
        output_path = os.path.join(self.model_dir, output_file)
        metadata.to_csv(output_path, index=False)
        print(f"\n✓ Results exported to: {output_path}")


def classify_new_pcap(pcap_file, model_name='lightgbm', export=True):
    """
    Main function to classify a new PCAP file
    
    Args:
        pcap_file: Path to PCAP file to analyze
        model_name: Which model to use ('sgd', 'lightgbm', 'xgboost')
        export: Whether to export results to CSV
        
    Returns:
        dict: Classification results
    """
    # Initialize inference engine
    engine = InferenceEngine()
    
    # Load models
    if not engine.load_models():
        print("✗ Failed to load models. Ensure training has been completed.")
        return None
    
    # Extract features from new PCAP
    features_dir = engine.extract_features(pcap_file)
    
    if not features_dir:
        print("✗ Failed to extract features")
        return None
    
    # Classify flows
    predictions, metadata = engine.classify_flows(features_dir, model_name)
    
    if predictions is None:
        print("✗ Classification failed")
        return None
    
    # Analyze results
    attack_counts = engine.analyze_results(predictions, metadata)
    
    # Export if requested
    if export:
        engine.export_results(metadata)
    
    # Clean up temporary features
    if Config.CLEANUP_TEMP:
        import shutil
        shutil.rmtree(features_dir)
        print("\n✓ Cleaned up temporary files")
    
    return {
        'total_flows': len(predictions),
        'attack_distribution': dict(attack_counts),
        'metadata': metadata
    }


# ========== USAGE EXAMPLE ==========
print("="*60)
print("INFERENCE MODE READY")
print("="*60)
print("\nTo classify a new PCAP file:")
print("1. Ensure you've completed a training run")
print("   (Models and features are saved automatically)")
print("2. Run classification on new PCAP:")
print("")
print("   # Example for Thursday's PCAP:")
print("   results = classify_new_pcap(")
print("       pcap_file='path/to/thursday.pcap',")
print("       model_name='lightgbm',  # or 'sgd'")
print("       export=True")
print("   )")
print("")
print("The function will:")
print("- Extract features from the new PCAP")
print("- Apply the trained model")
print("- Show attack distribution")
print("- Export results to CSV")
print("="*60)